In [1]:
SOURCE = "../bbb3_glass_eval_111716.log"

In [2]:
""" Install instructions
Always run in jupyterlab

- sudo apt install nodejs # didn't work
- curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.38.0/install.sh | bash
- nvm install --lts
- sudo chmod a+rwx /usr/local/share/jupyter/lab/
- mkdir /usr/local/share/jupyter/lab/extensions
- jupyter labextension install @jupyter-widgets/jupyterlab-manager --debug # will need to chmod some stouff to make work

- jupyter nbextension enable --py widgetsnbextension
"""

" Install instructions\nAlways run in jupyterlab\n\n- sudo apt install nodejs # didn't work\n- curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.38.0/install.sh | bash\n- nvm install --lts\n- sudo chmod a+rwx /usr/local/share/jupyter/lab/\n- mkdir /usr/local/share/jupyter/lab/extensions\n- jupyter labextension install @jupyter-widgets/jupyterlab-manager --debug # will need to chmod some stouff to make work\n\n- jupyter nbextension enable --py widgetsnbextension\n"

In [3]:
_= """ 
To use this file:
For vulcan, set the moving_laser_scan rayPose to pose_t(0,0,0) so the dumped scans are always in robot frame.
If you have the mapper.cpp return false right after dumping, it also speeds up the dump drastically.
This file requires also decent dumped odoe

Then the dumped 
"""

Known bugs:
 - Code is inconsistent on whether scan period is the whole period or just for the active part of the scan (hack corrected)


In 112016 the gyro(or odom rotation? not sure) readings need to be multiplied by  0.78(+-0.02) to be accurate(not tested on other runs)

The intensity seems to fall off at a rate of 218.3 (unknown uncert) units per meter for the Hokuyo. Perhaps it reports in a log scale? Not normalized in any case.
Below two meters, the intensity is pretty weird. 2-10 m is where the intensity seems to be sane. Somewhere between 0.5 and 1.0 m seems to be where most things report peak reflectivity

Localization seems to be accurate to +-10 mrad.

In [4]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import os

In [5]:
# rayfile = os.path.join(SOURCE, "raw_lidar_dumps","rays","rays.rays")
# rayfile = os.path.join(SOURCE, "raw_lidar_dumps","rays","rays_moving_unfixed.rays")
# rayfile = os.path.join(SOURCE, "raw_lidar_dumps","rays","rays_moving_earlyts.rays")
# rayfile = os.path.join(SOURCE, "raw_lidar_dumps","rays","rays_moving_hiqual.rays")
# rayfile = os.path.join(SOURCE, "raw_lidar_dumps","rays","rays_manyparticle_highrecall.rays")

rayfile = "/home/anuran/repos/Vulcan/bin/rayDump.rays" # HACK

In [6]:
rays_struct = np.dtype(
    [
        ('sx', np.float32),
        ('sy', np.float32),
        ('ex', np.float32),
        ('ey', np.float32),
        ('r' , np.float32),
        ('th', np.float32),
        ('i',  np.uint16),
        ('pad',  np.uint16),
        ('idx',  np.int32),
        ('ts', np.float64), #remove for old ray file formats
      ])

# rays = np.memmap(rayfile,rays_struct, 'r',1)
rays0 = np.fromfile(rayfile,rays_struct)
# Clip rays0 to a multiple of 1081 rays, under the assumption that a scan is 1081 rays long
if len(rays0)%1081 != 0:
    print("WARNING: file is not made of complete scans!")
    print("Remainder:", len(rays0)%1081)
# For the new sensors, we will need to find a different way to split scans
rays0 = rays0[:int(len(rays0)//1081*1081)]

polar_rays_struct = np.dtype(
    [
        ('r' , np.float32),
        ('th', np.float32),
        ('i',  np.uint16),
        ('pad',  np.uint16),
        ('idx',  np.int32),
        ('ts', np.float64), #remove for old ray file formats
      ])



In [7]:
rays1=rays0[rays0['idx']==0]
rays2=rays0[rays0['idx']==1]

In [8]:
%matplotlib widget
# plt.imshow(np.diff(rays0['ts'].reshape(-1,1081)[:1000]-rays0['ts'][0],axis=0))
# d=(d+np.pi/2)%
# plt.plot(np.diff(rays0['th'].reshape(-1,1081)[:1000],axis=0)[:,100],'b,')
# plt.plot(rays1['ts'],rays1['th'],'b,')
nan_th=rays1['th']/(rays1['r']>1.0)/(rays1['r']<30.0)
woof=np.diff(rays1['th'].reshape(-1,1081))
dts=np.diff(rays1['ts'].reshape(-1,1081))
dth=np.diff(nan_th.reshape(-1,1081))

slope=dth/dts
slope=slope[np.abs(dth)<np.pi/2]
stopped=[]
rangle=[]
for row,rts in zip(nan_th.reshape(-1,1081),rays1['ts'].reshape(-1,1081)):
    use = np.unwrap(row[np.isfinite(row)])
    use_ts = rts[np.isfinite(row)]
    if len(use)==0:
        print('err')
        print(row)
        break
    slope = (use[-1]-use[0])/(use_ts[-1]-use_ts[0])
    off=np.mean(use-(use_ts-rts[0])*slope)
    stopped.append(row-(rts-rts[0])*slope-off)
    rangle.append((rts-rts[0])*slope+off)
stopped = np.vstack(stopped)
rangle = np.vstack(rangle)
flat_rangle = np.unwrap(rangle[:,0].ravel()*2)/2
# plt.plot(rays1['ts'][::1081],flat_rangle,'b,')
_=plt.hist(np.diff(np.diff(flat_rangle)/np.diff(rays1['ts'][::1081])),1000)
plt.gca().set_yscale("log")
# plt.plot((stopped.ravel()+np.pi)%(2*np.pi)-np.pi,'b,')
# expect_th = (rays1['ts']-rays1['ts'][0])*np.float64(np.median(slope[slope>0]))%(2*np.pi)
# plt.plot(rays1['ts'][rays1['r']>1],(expect_th-rays1['th'])[rays1['r']>1]%(2*np.pi),'b,')
# plt.plot(np.diff(flat_rangle)/np.diff(rays1['ts'][::1081]))
# plt.hist(slope[np.isfinite(np.abs(slope))],1000)

<ipython-input-8-e50cb4296749>:6: RuntimeWarning: divide by zero encountered in true_divide
  nan_th=rays1['th']/(rays1['r']>1.0)/(rays1['r']<30.0)
<ipython-input-8-e50cb4296749>:6: RuntimeWarning: invalid value encountered in true_divide
  nan_th=rays1['th']/(rays1['r']>1.0)/(rays1['r']<30.0)
/usr/lib/python3/dist-packages/numpy/lib/function_base.py:1273: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
<ipython-input-8-e50cb4296749>:12: RuntimeWarning: invalid value encountered in less
  slope=slope[np.abs(dth)<np.pi/2]
<ipython-input-8-e50cb4296749>:30: RuntimeWarning: invalid value encountered in true_divide
  _=plt.hist(np.diff(np.diff(flat_rangle)/np.diff(rays1['ts'][::1081])),1000)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/usr/lib/python3/dist-packages/numpy/lib/histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/usr/lib/python3/dist-packages/numpy/lib/histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


Odo loader:

In [9]:
%matplotlib widget
# TODO: transition to floating point timestamps, not microseconds
odofile = "/home/anuran/repos/Vulcan/bin/odoDump.rays" # HACK
odo_struct = np.dtype(
    [
        ('ts', np.float64),
        ('x', np.float32),
        ('y', np.float32),
        ('th', np.float32),
        ('pad' , np.float32),
        ('lx', np.float32),
        ('ly', np.float32),
      ])
def odosub(odo):
    fields = ['ts','x','y','th']
    for f in fields:
        odo[f]-=odo[0][f]
    

# rays = np.memmap(rayfile,rays_struct, 'r',1)
poses = np.fromfile(odofile,odo_struct)
imu= poses[0::3]
odo= poses[1::3]
lp = poses[2::3]
assert(np.all(np.diff(odo['ts'])>=0))
assert(np.all(np.diff(imu['ts'])>=0))
assert(np.all(np.diff(lp['ts'])>=0))
odo['th']=np.unwrap(odo['th'])

Differentiator Function

In [10]:
# A nice differentiator function that works way better than diff
def d_dt(t,x,dt,order=1):
    if order>1:
        x=d_dt(t,x,dt,order-1)
    if order==0:
        return x
    dt = dt/2
    lot = np.clip(t-dt, t[0], t[-1])
    hit = np.clip(t+dt, t[0], t[-1])
    assert(np.all(hit-lot > 0))
    lox = np.interp(lot,t,x)
    hix = np.interp(hit,t,x)
    return (hix-lox)/(hit-lot)

In [18]:
th_step = (np.pi/180/4)
print('th_step',th_step)

raysx = rays1.reshape(-1,1081)[500:1000].ravel()
def derivatives(rays):
    th_step = (np.pi/180/4)
    x = rays['ex']
    y = rays['ey']
    th = rays['th']
    dr_dx = (np.sqrt((rays['ex']-(rays['sx']+0.01))**2+(rays['ey']-(rays['sy']))**2) - rays['r'])*100
    dr_dy = (np.sqrt((rays['ex']-(rays['sx']+0.01))**2+(rays['ey']-(rays['sy']+0.01))**2) - rays['r'])*100
    dth_dx = (np.arctan2((rays['ey']-(rays['sy'])),(rays['ex']-(rays['sx']+0.01))) - rays['th'])*100/th_step # from radians to steps on the output
    dth_dy = (np.arctan2((rays['ey']-(rays['sy']+0.01)),(rays['ex']-(rays['sx']))) - rays['th'])*100/th_step # from radians to steps on the output
    x2_dth = rays['ex']*np.cos(th_step) + rays['ey']*np.sin(th_step) 
    y2_dth = -rays['ex']*np.sin(th_step) + rays['ey']*np.cos(th_step) 
    dth_dth = (np.arctan2(y2_dth-rays['sy'], x2_dth-rays['sx']) - rays['th'])/th_step/th_step # from radians to steps on the output
    dr_dth = (np.sqrt((y2_dth-rays['sy'])**2 + (x2_dth-rays['sx'])**2)-rays['r'])/th_step
    D = np.vstack((dr_dx.ravel(), dr_dy.ravel(), dr_dth.ravel(),
                   dth_dx.ravel(), dth_dy.ravel(), dth_dth.ravel()))
    D[:,rays['r'] < 0.05] = 0
    D[:,rays['r'] > 50] = 0
    D = D.reshape(2,3,-1)
    return D

D_r_th__x_y_th=derivatives(raysx) # dims: NxOutxIn

# Test it 
dx = np.interp(raysx['ts'], odo['ts'], d_dt(odo['ts'],odo['x'],1/40))/40
dy = np.interp(raysx['ts'], odo['ts'], d_dt(odo['ts'],odo['y'],1/40))/40
ds = dx*np.cos(np.interp(raysx['ts'], odo['ts'],odo['th']))+dy*np.sin(np.interp(raysx['ts'], odo['ts'],odo['th']))

dth = np.interp(raysx['ts'], odo['ts'], d_dt(odo['ts'],odo['th'],1/40))/40
# print(np.min(ds))
print(2*np.max(ds))
print(2*np.max(dth)/th_step)

print(D_r_th__x_y_th.shape)
motions = np.vstack((ds.ravel(),np.zeros_like(ds),dth.ravel()))
D_r_th=np.sum(D_r_th__x_y_th*motions[np.newaxis, :, :],axis=1)

# D_r_th = np.dot(D_r_th__x_y_th, np.vstack((dx.ravel(),dy.ravel(),dth.ravel())))
# D_r_th = np.tensordot(D_r_th__x_y_th,  np.vstack((dx.ravel(),dy.ravel(),dth.ravel())), 1)
# D_r_th = np.einsum('Noi,iN->No',D_r_th__x_y_th,  np.vstack((dx.ravel(),dy.ravel(),dth.ravel())))
print(D_r_th.shape)
%matplotlib widget
raysx=raysx.reshape(-1,1081)
# plt.plot(raysx['r'][1:].ravel()[7907531:7907531+2162],'b.')
# plt.plot(raysx['r'].ravel()[7907531:7907531+2162],'r.')
# plt.plot(np.arange(2162)+D_r_th[1][7907531:7907531+2162],raysx['r'].ravel()[7907531:7907531+2162]+D_r_th[0][7907531:7907531+2162],'g.')
# plt.plot(np.arange(2162)+D_r_th[1][7907531:7907531+2162]*2,raysx['r'].ravel()[7907531:7907531+2162]+D_r_th[0][7907531:7907531+2162]*2,'y.')

th_step 0.004363323129985824
0.11017027127612611
8.969295515952949
(2, 3, 540500)
(2, 540500)


In [15]:
# D_r_th = D_r_th.reshape(2,-1,1081)[:,:384].copy()
r=raysx['r'].copy().reshape(-1,1081)[:,:384].copy()
D_r_th__x_y_th_b=D_r_th__x_y_th.reshape(2,3,-1,1081)[...,:384].copy()
def score_motions(r, D_r_th__x_y_th_b, smin, smax, ssteps, thmin, thmax, thsteps):
    N=r.shape[1]
    moxies=[]
    for sinc in np.linspace(0,.15,16,dtype=np.float32):
        for thinc in np.linspace(-10,10,41,dtype=np.float32)*th_step:
            print('.',end=' ')
            d_r  = D_r_th__x_y_th_b[0,0]*sinc+D_r_th__x_y_th_b[0,2]*thinc
            d_th = D_r_th__x_y_th_b[1,0]*sinc+D_r_th__x_y_th_b[1,2]*thinc
            thoff = np.clip(d_th+np.arange(N)[np.newaxis,:],0,N-1)[:-1]
            thlo = thoff.astype('int16')
            thhi = np.ceil(thoff).astype('int16')
            rlo = r[1:][np.arange(len(r)-1)[:, np.newaxis], thlo]
            rhi = r[1:][np.arange(len(r)-1)[:, np.newaxis], thhi]
            rlin = (1-thoff%1)*rlo+(thoff%1)*rhi
    #         print(r.shape)
            moxies.append(np.sum(np.clip(np.abs(rlin - (r[:-1]+d_r[:-1])),-0.05,0.05),axis=1))
    return moxies

r = raysx['r'].copy().reshape(-1,1081)[:, :384].copy()
D_r_th__x_y_th_b = D_r_th__x_y_th.reshape(2,3,-1,1081)[..., :384].copy()
ma = score_motions(r, D_r_th__x_y_th_b, smin=0, smax=0.15, ssteps=16, thmin=-10, thmax=10, thsteps=41)

r = raysx['r'].copy().reshape(-1,1081)[:, 384:768].copy()
D_r_th__x_y_th_b = D_r_th__x_y_th.reshape(2,3,-1,1081)[..., 384:768].copy()
mb = score_motions(r, D_r_th__x_y_th_b, smin=0, smax=0.15, ssteps=16, thmin=-10, thmax=10, thsteps=41)

r = raysx['r'].copy().reshape(-1,1081)[:, 768:].copy()
D_r_th__x_y_th_b = D_r_th__x_y_th.reshape(2,3,-1,1081)[..., 768:].copy()
mc = score_motions(r, D_r_th__x_y_th_b, smin=0, smax=0.15, ssteps=16, thmin=-10, thmax=10, thsteps=41)
  

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [30]:
%matplotlib widget
readout=np.hstack((np.array(ma).T.reshape(-1,41),np.array(mb).T.reshape(-1,41),np.array(mc).T.reshape(-1,41)))
A=np.array(ma).T.reshape(-1,16,41)
plat=np.argmin(A,axis=2).ravel()
B=np.array(mb).T.reshape(-1,16,41)
platb=np.argmin(B,axis=2).ravel()
C=np.array(mc).T.reshape(-1,16,41)
platc=np.argmin(C,axis=2).ravel()

# readout = np.array(mb).T.reshape(-1,41)
# np.rollaxis(readout)
# help(np.swapaxes)
# print(np.array(moxies).shape)
plt.imshow(readout,aspect='auto')
plt.plot(plat,np.arange(len(plat)),'r')
plt.plot(platb,np.arange(len(plat)),'k')
plt.plot(platc,np.arange(len(plat)),'c')

# plt.plot(np.diff(np.interp(raysx['ts'][:,540],odo['ts'],odo['th']))/th_step)
#         rhi
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
a=1.0
while(np.float32(a/2+8.0)!=8.0):
    a/=2
print(a, 1/2**20)

9.5367431640625e-07 9.5367431640625e-07


In [424]:
# Plan: 
# 1. Project scans back by one to solve for best time offset from the surrounding ones
# 2. Average offsets within a block to get new pseudo times for each block
# 3. Assign pseudotimes
# 4. Repeat
cos=np.cos; sin = np.sin
odo['th']=np.unwrap(odo['th'])
def in_base_frame(s, odo):
    out = s.copy()
    xo=np.interp(s['ts'],odo['ts'],odo['x'])
    yo=np.interp(s['ts'],odo['ts'],odo['y'])
    tho=np.interp(s['ts'],odo['ts'],odo['th'])
    x=s['ex']
    y=s['ey']
    xp = cos(tho)*x - sin(tho)*y + xo
    yp = sin(tho)*x + cos(tho)*y + yo
    out['ex'] = xp
    out['ey'] = yp
    x=s['sx']
    y=s['sy']
    xp0 = cos(tho)*x - sin(tho)*y + xo
    yp0 = sin(tho)*x + cos(tho)*y + yo
    out['sx'] = xp0
    out['sy'] = yp0
    gts = np.arange(len(out['ts']))[out['r']>0]
    ang = np.arctan2(yp-yp0,xp-xp0)
    gang = np.unwrap(ang[out['r']>0])
    out['th'] = np.interp(np.arange(len(out['ts'])), gts, gang)
    out['r'][out['r']>0]= np.sqrt((xp-xp0)**2+(yp-yp0)**2)[out['r']>0]
    return out
    
def get_err(s1,s2):#result is at sample points of s2
    MAX_ERR=0.2
    good = (s1['r']>0 )& (s1['r']<40)
    xc=s1['sx'][540]
    yc=s1['sy'][540]
    th = np.arctan2(s2['ey']-yc, s2['ex']-xc)%(2*np.pi) # TODO: figure out which sx and sy are the right ones to pair!
    r = np.sqrt((s2['ey']-yc)**2+(s2['ex']-xc)**2)
    
    th1= np.arctan2(s1['ey']-yc, s1['ex']-xc)%(2*np.pi)
#     r1 = np.sqrt((s2['ey']-yc)**2+(s2['ex']-xc)**2) #not used, but could prob simplify lookup
    
#     plt.plot(th%(2*np.pi),r)
    
    errx = np.interp(th, th1, s1['ex'], period=2*np.pi)-xc
    erry = np.interp(th, th1, s1['ey'], period=2*np.pi)-yc
    r_from_s1=np.sqrt(errx**2+erry**2)
    err=r_from_s1-r # TODO: dot product with central direction instead of sqrt a square, so we have sign of error
#     plt.plot(th, np.sqrt(errx**2+erry**2),'.')
#     err = errx*np.cos(th)+erry*np.sin(th)
    
    err*=(s2['r']>0)&(s2['r']<40)
    err = np.clip(err, -MAX_ERR, MAX_ERR)
    return err

# def project(ss, ns, odo):
#     # this version is pretty bad
#     ssb=in_base_frame(ss, odo)
#     nsb=in_base_frame(ns, odo)
#     nsmin = ns.copy(); nsmin['ts']-=4/40
#     nsbmin = in_base_frame(nsmin, odo)
#     nsmax = ns.copy(); nsmax['ts']+=4/40
#     nsbmax = in_base_frame(nsmax, odo)
#     derr_dst = (get_err(ssb,nsbmax)-get_err(ssb,nsbmin))/8 #error per scan displaced
#     err = get_err(ssb,nsb)
#     offset = (derr_dst.dot(err))/derr_dst.dot(derr_dst) 
#     return offset, np.clip(err/derr_dst,-4,4)*err

def project(ss, ns, odo, noffsets=17):
    ssb=in_base_frame(ss, odo)
    nsb=in_base_frame(ns, odo)
    
    offsets = np.linspace(-4/40,4/40,noffsets) if noffsets>1 else np.array([0])
#     plt.plot(ssb['ex'],ssb['ey'],'.')
#     plt.plot(ssb['th'][ssb['r']<40],ssb['r'][ssb['r']<40],'.')
    stack=[]
    for off in offsets:
        nsoff = ns.copy(); nsoff['ts']+=off
        nsboff = in_base_frame(nsoff, odo)
#         plt.plot(nsboff['th'][nsboff['r']<40],nsboff['r'][nsboff['r']<40],)
        err = get_err(ssb,nsboff)
#         plt.plot(nsboff['th'],nsboff['r']+err,'x')
#         plt.plot(nsboff['th'], err)
        stack.append(err)
    
    stack = np.array(stack)
    minerridx = np.argmin(np.abs(stack),axis=0)
    minerr = stack[minerridx,np.arange(len(ns))]
    
    sols=offsets[minerridx] # sols is the best offsets
    residual=np.min(np.abs(stack),axis=0) #qual is the abs err at the min offset
    
    nsoff = ns.copy(); nsoff['ts']+=sols+1/40
    nsboff = in_base_frame(nsoff, odo)
    errplus = get_err(ssb,nsboff)
    nsoff['ts']-=2/40
    nsboff = in_base_frame(nsoff, odo)
    errminus = get_err(ssb,nsboff)
    sensitivity = np.minimum(np.abs(errminus-minerr),np.abs(errplus-minerr))

#     qual=np.min(np.abs(stack),axis=0)
    return sols*40, sensitivity, minerr

def resolve_times(pseudotime):
    offset=tsj[0]*40
    resolved = []
    for ps in pseudotime:
        apptime = ps-offset
        offset+= ((apptime+0.5)%1-0.5)/10
        resolved.append(int(np.round(apptime)))
        
    return resolved

                             
                             
# %matplotlib widget
# raysx=rays2.copy().reshape(-1,1081)
raysx=raysy.copy()#HACK! remove for original style
# raysx[356:371,768:]['ts']-=1/40

# raysx[150:298,:384]['ts']-=1/40
# raysx[320:323,:384]['ts']-=1/40
# raysx[450:540,:384]['ts']-=1/40
# raysx[454:502,:384]['ts']+=1/40
# raysx[463:478,:384]['ts']-=1/40
# raysx[474,:384]['ts']+=1/40
# raysx[488,:384]['ts']-=1/40
# raysx[491,:384]['ts']-=1/40
# raysx[521:531,:384]['ts']+=1/40
# raysx[523,:384]['ts']-=1/40
# raysx[536:539,:384]['ts']-=1/40
# raysx[710:740,:384]['ts']+=1/40
# raysx[732,:384]['ts']-=1/40
# raysx[737:740,:384]['ts']-=1/40
# raysx[745:750,:384]['ts']+=1/40
# raysx[746,:384]['ts']-=1/40
# raysx[808:847,:384]['ts']+=1/40
# raysx[927:939,:384]['ts']+=1/40
# raysx[951:954,:384]['ts']+=1/40
# raysx[955:960,:384]['ts']+=1/40
# raysx[992:1000,:384]['ts']-=1/40

# raysx[668:992,:768]['ts']-=1/40

# raysx[643:649,384:768]['ts']-=1/40
# raysx[643:646,384:768]['ts']+=1/40

# raysx[646,384:768]['ts']+=1/40
# raysx[647:649,384:768]['ts']+=1/40

# raysx[739:750,384:768]['ts']+=1/40


# raysx[520,384:]['ts']-=1/40

# raysx[137:141,768:]['ts']+=1/40
# raysx[146:356,768:]['ts']+=1/40
# raysx[356:371,768:]['ts']-=1/40
# raysx[382:413,768:]['ts']+=1/40
# raysx[453,768:]['ts']+=1/40
# raysx[491:497,768:]['ts']+=1/40
# raysx[504:506,768:]['ts']+=1/40
# raysx[520:531,768:]['ts']+=1/40
# raysx[633:635,768:]['ts']+=1/40
# raysx[630:684,768:]['ts']=rays2.reshape(-1,1081)[630:684,768:]['ts']
# raysx[638,768:]['ts']+=1/40
# raysx[641,768:]['ts']+=1/40
# raysx[645,768:]['ts']+=1/40
# raysx[684:708,768:]['ts']-=1/40
# raysx[708:740,768:]['ts']+=1/40
# raysx[738:739,768:]['ts']-=1/40
# raysx[750:756,768:]['ts']-=1/40
# raysx[890:1000,768:]['ts']+=1/40
# raysx[903:922,768:]['ts']-=1/40
# raysx[959:961,768:]['ts']-=1/40
# raysx[973:977,768:]['ts']-=1/40
# raysx[984:993,768:]['ts']-=1/40



tsj = raysx[:,0]['ts'].copy()
pseudotime=tsj*40
resolved = resolve_times(pseudotime)

solvate = []
qual = []
residual = []
ans = []
for scannum in range(0,len(raysx)):
#     print(raysx[0]['r'][raysx[0]['r']>0])
    sol, q, resid = project(raysx[scannum-1], raysx[scannum], odo,41)
#     assert np.all(sol==0)
    s=sol
    qf = np.abs(q/(np.abs(resid)+0.000001))
    slc=slice(0,384)
    A = round(np.median(s[slc][(np.abs(q[slc])>0.01)&(qf[slc]>4)]),2)
    slc=slice(384,768)
    B = round(np.median(s[slc][(np.abs(q[slc])>0.01)&(qf[slc]>4)]),2)
    slc=slice(768, None)
    C = round(np.median(s[slc][(np.abs(q[slc])>0.01)&(qf[slc]>4)]),2)
#     Y = resolved[scannum]-resolved[scannum-1]
#     print(A,B,C,Y)
    ans.append((A,B,C))
#     print(np.mean(q[(q<0.05)&(qf>4)]))
#     assert scannum<=2 or 0 < Y <= 2
#     buff=[]
#     for j in range(5):
#         q, sol = project(raysx[scannum-j], raysx[scannum], odo)
#         buff.append(sol)
#     sol=np.mean(buff,axis=0)
    solvate.append(sol)
    qual.append(q)
    residual.append(resid)
    print(scannum, end=' ')


# # # plt.clf()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [426]:
%matplotlib widget
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
ans = np.array(ans)
solvate=np.array(solvate)
# solvate[:,16:30]=np.median(solvate[:,0:384],axis=1)[:,np.newaxis]
mask = (np.abs(qual)>0.02)&(np.abs(solvate)<=3.5)
qual_frac = qual/(np.abs(residual)+.00001)
solve2=(solvate+.2)*(qual_frac>4)*(np.abs(qual)>0.01)-0.2#*np.clip(qual,-.01,.01)*100
# solve2[:,260:312]=ans[:,1:2]
# solve2[np.abs(solve2)>3] = -4
# solve2[:,9:16]= (np.mean(np.abs(solve2),axis=1)/(np.mean(mask,axis=1)+.001))[:,np.newaxis]
blockA=np.array([np.median(s[0:384][np.abs(q[:384])>0.02])for s,q in zip(solvate,qual)])
blockA[np.isnan(blockA)]=0
# solve2[:,16:30]=blockA[:,np.newaxis]
cm=tuple(np.linspace(0,1,3000).reshape(-1,1)*[1,1,1])
cm=ListedColormap(cm)
plt.imshow(np.clip(1/raysx[500:1000]['r'],0,3),aspect = 'auto', cmap=cm)
plt.figure()
plt.imshow(solve2, aspect = 'auto', interpolation='nearest', cmap='bwr')
plt.figure()
plt.imshow((raysx['ts']-rays2['ts'].reshape(-1,1081)[:len(raysx)])[:len(solve2)], aspect = 'auto', interpolation='nearest', cmap='bwr')
plt.figure()
plt.imshow(np.clip(qual/(np.abs(residual)+.00001),0,2), aspect = 'auto',cmap='bwr')
plt.figure()
plt.imshow(residual, aspect = 'auto',cmap='bwr')

plt.imsave("Corrected_residuals.png", solve2,cmap='bwr')
# plt.plot([np.median(s[0:384][np.abs(s[0:384])<2])for s in solvate])

#notes: Block A likes to go -1 or 0. Block B likes to go 0 or -1. Block C likes to go 0 or +1

# First, subtract off one scan period from ts on A and B blocks for their base value. Set their starting offset to 1 scan.
# For block C just set the starting offset to 0. When trying out options, just make sure to keep the offset in %2 and add the appropriate time to the base

# If chose a wrong block while spinning, will see a diagonal across the 384 point(or other block crossing!)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
%matplotlib widget

tsj = raysx[:,0]['ts'].copy()
plt.plot(np.diff(tsj)*40,'b.')

pseudotime=tsj*40
def resolve_times(pseudotime):
    offset=tsj[0]*40
    resolved = []
    for ps in pseudotime:
        apptime = ps-offset
        offset+= ((apptime+0.5)%1-0.5)/10
        resolved.append(int(np.round(apptime)))
        
    return resolved
resolved = resolve_times(pseudotime)
plt.plot(np.diff(resolved))

newview = np.zeros((resolved[-1]+2, 1081))-3
for outidx, inidx in zip(resolved, range(len(solvate))):
    newview[outidx] = solve2[inidx]
plt.imshow(newview[:outidx+1], aspect = 'auto', cmap='bwr')



# Energy: For every pair of blocks
# F(a_block, b_block, excess_jump) is the energy function
# for a trial between two scans:
# off1, off2, off3
# off4, off5, off6
# we get 

stub=np.zeros(1081)
def score(s1, s2, offsets):
    # returns the cost for a given choice of scan offsets
    if np.any(offsets)<-1 or np.any(offsets)>1:
        return -100, stub
    if np.any(np.abs(offsets[3:]-offsets[:3])>1):
        return -100, stub
    if np.any(offsets*[-1,-1,1,-1,-1,1] < 0): #we know what signs are allowed now, so suppress the others
        return -100, stub
    s1[0:384]['ts']+=offsets[0]/40
    s1[384:768]['ts']+=offsets[1]/40
    s1[768:]['ts']+=offsets[2]/40
    s2[0:384]['ts']+=offsets[3]/40
    s2[384:768]['ts']+=offsets[4]/40
    s2[768:]['ts']+=offsets[5]/40
    sol, q, resid = project(s1, s2, odo,9)
    assert(len(resid)==1081)
    qf = np.abs(q/(np.abs(resid)+0.000001))
    good = (q>0.01)&(qf>4)&(np.abs(sol)<=1)
    
    scores= good*(sol==0)*np.clip(q,-0.05,0.05)
#     print(np.sum(scores>=0))
#     scores=(sol[good]==0)*np.clip(q[good],-0.05,0.05) # minimize the residual on reliable samples # HIGH GOOD
#     scores=(sol==0)*np.clip(resid,-0.05,0.05) # minimize the residual on reliable samples # LOW GOOD
    score = np.sum(np.abs(scores))
    s1[0:384]['ts']-=offsets[0]/40
    s1[384:768]['ts']-=offsets[1]/40
    s1[768:]['ts']-=offsets[2]/40
    s2[0:384]['ts']-=offsets[3]/40
    s2[384:768]['ts']-=offsets[4]/40
    s2[768:]['ts']-=offsets[5]/40
    return score, scores

def solve(s1,s2):
    lsc=np.zeros(1081)
    off=np.zeros(1081)
    loff = -.02*np.zeros(1081)
    
    sols = {}
    for o1 in range(-1,2):
        for o2 in range(-1,2):
            for o3 in range(-1,2):
                for o4 in range(-1,2):
                    for o5 in range(-1,2):
                        for o6 in range(-1,2):
                            offsets = (o1,o2,o3,o4,o5,o6)
                            sols[offsets], scores = score(s1, s2, np.array(offsets))
                            
                            #debug
                            better=scores>lsc
                            off[0:384] = o4-o1
                            off[384:768] = o5-o2
                            off[768:] = o6-o3
                            lsc[better]=scores[better]
                            loff[better]=off[better]
    return sols, lsc, loff

def vitterbi(sn,bps,bcs,img):
    states=[]
    for o1 in range(-1,2):
        for o2 in range(-1,2):
            for o3 in range(-1,2):
                states.append((o1,o2,o3))
                
    curr_cost = bcs[-1] if len(bcs) else [0]*len(states)
    startstep = sn[-1]+1 if sn else 0
    for stepnum in range(sn[-1]+1,len(raysx)-1):
        print(stepnum)
        print(curr_cost)
        next_cost = [-1]*len(states)
        best_prior = [0]*len(states)
        s1 = raysx[stepnum]
        s2 = raysx[stepnum+1]
        sols, lsc, loff= solve(s1,s2)
#         print("Best:", list(sols.items())[np.argmax(list(sols.values()))])
#         print(sols)
#         print(sum(lsc>0))
        img.append(lsc*loff-0.1*(lsc==0))
        for startidx in range(len(states)):
            for endidx in range(len(states)):
                offsets = states[startidx]+states[endidx]
                newcost = curr_cost[startidx]+sols[offsets]
                oldcost = next_cost[endidx]
                if newcost>oldcost:
                    best_prior[endidx] = startidx
                    next_cost[endidx] = newcost
                    
                if newcost==oldcost:
                    if np.sum(np.abs(states[startidx]))<=np.sum(np.abs(states[best_prior[endidx]])):
                        best_prior[endidx] = startidx
                        next_cost[endidx] = newcost
        print(best_prior)
        sn.append(stepnum)
        bps.append(best_prior)
        bcs.append(next_cost)
        curr_cost = next_cost
    return bps, bcs, img
            
# bps = []
# bcs = []
# sn = [-1]
# img = []
# bcs = list(bcs)
bps,bcs,img = vitterbi(sn,bps,bcs,img)

In [420]:
# np.savez('vitterbi_intermediates_2state.npz', original_rays = raysx, retimed_rays = raysy, bcs=bcs, bps=bps, img=img, sn = sn)

In [419]:
# Decode the vitterbi outpus
states=[]
for o1 in range(-1,2):
    for o2 in range(-1,2):
        for o3 in range(-1,2):
            states.append((o1,o2,o3))
%matplotlib widget
# print(bps)
# plt.imshow(bcs)
# solve(raysx[177],raysx[178])
start=np.argmax(bcs[-1])
print(states[start])
solution=[]
sol_idx=[]
for line in reversed(bps):
#     print(states[start])
    sol_idx.append(start)
    solution.append(states[start])
    start = line[start]
#last iteration is needed since we got from t to t+1, so the first state is pointed to by the initial bps
sol_idx.append(start) 
solution.append(states[start]) # the last solution is needed


sol_idx = np.array(list(reversed(sol_idx)))
solution = np.array(list(reversed(solution)))
# bcs=np.array(bcs)
# cost_path = bcs[np.arange(len(sol_idx)),sol_idx]
delta_cost = np.diff(cost_path, prepend=0)
pre= np.diff(solution,axis=0,prepend=0)
# help(np.diff)
# print(solution)
dimg=np.vstack((np.zeros(1081),np.array(img).copy()))# zeros because first row not availale
bkg=dimg.copy()
bkg[:,:384]=pre[:,0:1]*0.05
bkg[:,384:768]=pre[:,1:2]*0.05
bkg[:,768:]=pre[:,2:3]*0.05
dimg[dimg==-0.1]=bkg[dimg==-0.1]*0.5
plt.imshow(dimg,aspect='auto')
plt.figure()
plt.imshow(solution,aspect='auto')
dth_dt=d_dt(odo['ts'],odo['th'],1/20)
w = np.interp(raysx['ts'][:,-1], odo['ts'], dth_dt )
plt.plot(w[:len(solution)],np.arange(len(solution)))
plt.plot(delta_cost/10,np.arange(len(solution)))
plt.figure()
plt.plot(w[:len(solution)],delta_cost,'b.')

(-1, -1, 0)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: x and y must have same first dimension, but have shapes (1,) and (8642,)

In [421]:
%matplotlib widget
# bcs=np.array(bcs)
# bcs[sn]=bcs
# bps=np.array(bps)
# bps[sn]=bps
# sn=np.array(sn)
# sn[sn]=sn
raysy=raysx[:len(solution)].copy()
raysy[:,:384]['ts']+=solution[:,0:1]*1/40
raysy[:,384:768]['ts']+=solution[:,1:2]*1/40
raysy[:,768:]['ts']+=solution[:,2:3]*1/40
# plt.plot(np.diff(sn))
# plt.imshow(raysy['ts']-raysx[:len(solution)]['ts'])
# plt.plot(rays0[::1081]['ts'],rays0[::1081]['idx']==1,'b.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [430]:
# inspect the quality of our fix
# Project the scan to the base frame, and then to the times they should have been
def out_base_frame(s,ts,odo):
    # does the opposite of in_base_frame (sorry! couldn't think of good name)
    # but uses the specified times instead of the ones in the scan to define the local frame
    out = s.copy()
    xo=np.interp(ts,odo['ts'],odo['x'])
    yo=np.interp(ts,odo['ts'],odo['y'])
    tho=np.interp(ts,odo['ts'],odo['th'])
    x=s['ex']
    y=s['ey']
    xp = cos(tho)*(x-xo) + sin(tho)*(y-yo)
    yp = -sin(tho)*(x-xo) + cos(tho)*(y-yo)
    out['ex'] = xp
    out['ey'] = yp
    x=s['sx']
    y=s['sy']
    xp0 = cos(tho)*(x-xo) + sin(tho)*(y-yo)
    yp0 = -sin(tho)*(x-xo) + cos(tho)*(y-yo)
    out['sx'] = xp0
    out['sy'] = yp0
    gts = out['ts'][out['r']>0]
    ang = np.arctan2(yp-yp0,xp-xp0)
    gang = np.unwrap(ang[out['r']>0])
    out['th'] = np.interp(out['ts'], gts, gang)
    out['r'][out['r']>0]= np.sqrt((xp-xp0)**2+(yp-yp0)**2)[out['r']>0]
    return out
    
# %matplotlib widget

scn = 136
rx_bf = in_base_frame(raysx[scn],odo)
ry_bf = in_base_frame(raysy[scn],odo)
ts = raysx[scn]['ts'] #use the expected time
s1 = out_base_frame(rx_bf,raysx[scn]['ts'],odo) #should be identical up to roundoff from the original
s2 = out_base_frame(ry_bf,raysx[scn]['ts'],odo)
plt.plot(s1['ex'],s1['ey'], 'b.')
plt.plot(s2['ex'],s2['ey'], 'r.', markersize=1)

# raysy=raysx #HACK
# raysx=rays2.copy().reshape(-1,1081)#end HACK

from ipywidgets import interact, interact_manual
plt.plot([1],[1])
@interact(scn=(0,len(raysy)-1,1))
def draw(scn=7280):
    for artist in plt.gca().lines + plt.gca().collections:
            artist.remove()
    rm_bf = in_base_frame(raysx[scn-1],odo)
    rx_bf = in_base_frame(raysx[scn],odo)
    ry_bf = in_base_frame(raysy[scn],odo)
    ts = raysx[scn]['ts'] #use the expected time
    s1 = out_base_frame(rx_bf,raysx[scn]['ts'],odo) #should be identical up to roundoff from the original
    s2 = out_base_frame(ry_bf,raysx[scn]['ts'],odo)
#     plt.plot(s1['ex'],s1['ey'], 'b.')
#     plt.plot(s2['ex'],s2['ey'], 'r.', markersize=2)
#     plt.plot(rm_bf['ex'],rm_bf['ey'], 'kx')
    plt.plot(rx_bf['ex'][:384],rx_bf['ey'][:384], 'gx',markersize=10)
    plt.plot(rx_bf['ex'][384:768],rx_bf['ey'][384:768], 'go')
    plt.plot(rx_bf['ex'][768:],rx_bf['ey'][768:], 'g.')
#     plt.plot(ry_bf['ex'],ry_bf['ey'], 'r.', markersize=2)
#     plt.clf()
    tmp1=raysx[scn].copy();tmp2=raysx[scn].copy()
    tmp1['ts']-=1/40;tmp2['ts']+=1/40
    tmp1=in_base_frame(tmp1,odo);tmp2=in_base_frame(tmp2,odo)
    qx=np.vstack((tmp1['ex'],rx_bf['ex'],rx_bf['ex']*np.nan))
    qy=np.vstack((tmp1['ey'],rx_bf['ey'],rx_bf['ey']*np.nan))
    plt.plot(qx.T.ravel(),qy.T.ravel(),'b')
    qx=np.vstack((tmp2['ex'],rx_bf['ex'],rx_bf['ex']*np.nan))
    qy=np.vstack((tmp2['ey'],rx_bf['ey'],rx_bf['ey']*np.nan))
    plt.plot(qx.T.ravel(),qy.T.ravel(),'r')
    for i in range(-9,1):
        ry_bf = in_base_frame(raysy[scn+i],odo)
        plt.plot(ry_bf['ex'],ry_bf['ey'], '.',mew=0, markersize=10+i, mfc=[(10+i)/10,0,(-i)/10])
    plt.title(str(solution[scn]))
    
# np.savez("hand_retimed_1000", rays2=rays2, retimed=raysy, odometry=odo)

interactive(children=(IntSlider(value=7280, description='scn', max=8641), Output()), _dom_classes=('widget-int…

In [437]:
fig = plt.figure()
plt.plot(#np.unwrap(raysy['th'].ravel()),
         rays2['ts'].ravel(),'r,')
plt.plot(#np.unwrap(raysy['th'].ravel()),
         raysy['ts'].ravel(),'b,')
# plt.plot(#np.unwrap(raysy['th'].ravel()),
#          raysy['ts'].ravel()+1/40,'r,')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [549]:
# faster way to do this crap: 
# 1. Project everything to the base frame and save it
# 2. Project everything -1 and +1 timestep to the base frame and also save that
# 3. Take the points in 2 and measure them against the linear interpolant of 1. You now have 3 numbers per point, and associated block couplings
# 3.5 Discard couplings that are worse than a certain threshold
# 4. Now you have evidence for hi, lo, or same as last time
# 5. Put those in a big matrix equation for the time of every block and solve it? You can have the target be the solution picked for that point, and the gradient be the quality
# 6. 
%matplotlib widget
# plt.plot(raysx[500:600]['th'].T,raysx[500:600]['r'].T,',')
# plt.hist(rays0['r'],500, weights=np.abs(rays0['r']))
# help(plt.hist)
plt.imshow
rsteps = 50
roto = np.zeros((rsteps*2,1081), np.int64)
r = raysx['r']
stripes=np.abs((r/0.99*255).astype(np.int8)).astype(np.uint8)*2
# plt.imshow(np.clip(np.diff(r,axis=0), -0.05,0.05
#                   ),aspect='auto',cmap='gray')
print(np.sum(np.diff(r,axis=0)==0)/384)
# rotation rate up to 50 spots per scan
print(len(r), 3*len(r))
print(np.sum(solution!=0))


def build_differentials(rays,odo):
    # requires 0.2sec per invocation on 8k scans
    # probabably mostly in the giant interp calls
    
    # returns the velocity of the ray endpoints wrt the coordinate frame of the bot
    x = rays['ex']*np.sign(rays['r'])
    x.reshape(-1)[np.sign(rays['r'].ravel())<0] = (-np.cos(rays['th'])*rays['r']).reshape(-1)[np.sign(rays['r'].ravel())<0]
    y = rays['ey']*np.sign(rays['r'])
    y.reshape(-1)[np.sign(rays['r'].ravel())<0] = (-np.sin(rays['th'])*rays['r']).reshape(-1)[np.sign(rays['r'].ravel())<0]
#     print(odo.dtype)
    dx = d_dt(odo['ts'],odo['x'],1/10)
    dy = d_dt(odo['ts'],odo['y'],1/10)
    dx_dt = dx*np.cos(odo['th'])+dy*np.sin(odo['th']) # change to x from bot coords instead of x from world coords
    dth_dt = d_dt(odo['ts'],odo['th'],1/10)
    dx_dt = np.interp(rays['ts'],odo['ts'],dx_dt)
    dth_dt = np.interp(rays['ts'],odo['ts'],dth_dt)
    dx = dx_dt*np.sign(rays['r'])-dth_dt*y
    dy = dth_dt*x
    return dx, dy

# motion model: 
rinv = 1/raysx['r']
def build_derivatives(r,th):
    #       fwd        turn    slip
    # dr    sin/r        0      -sin
    # dth   -cos        -1      -cos/r
    
    # [1/r 1]  sin      0       0        fwd
    #           0       0       -sin  @  turn
    #                                    slip
    
    # dr  =  1/r* (fwd*sin) + (-sin*slip)
    3
    
    dr_dfwd = -cos(th)
    dth_dfwd = sin(th)/r
    dr_dturn = 0
    dth_dturn = -1
    return 
    
# we will need +-12 angle bins to cover +- 2m/s at 40hz
# assuming we only use things >1m away and max speed is 4m/s, we can see up to 0.1m of motion at 40 hz, or an additional +- 23 bins
# so true max is +-35 bins at 40 hz, or +-70 bins at 20hz when we miss some scans
# for those offsets, we can have up to 0.2m of range change
# Use scans 2891 and 2892 as examples for testing

print("max motion",np.max(d_dt(odo['ts'],odo['th'],1/20)))

import timeit
help(timeit.timeit)
dx,dy = build_differentials(raysx,odo)
# print(timeit.timeit(l,number=100))
img=(np.diff(np.abs(r),axis=0))
expect=np.sqrt(dx**2+dy**2)[:-1]/40
# plt.imshow(expect,aspect='auto',cmap='gray')
# plt.imshow(np.clip(img,0,0.1),aspect='auto',cmap='gray')
plt.imshow(np.clip(img/np.maximum(expect,0.005),-2.5,2.5),aspect='auto',cmap='gray') #normalized to expected motion magnitude

# plt.imshow(np.clip(1/np.abs(raysx['r']),0,1),aspect='auto',cmap='gray')

aug = np.hstack((np.clip(np.abs(np.diff(r,axis=0)), -0.05,0.05), np.sqrt(dx**2+dy**2)[:-1]/40))
# plt.imshow(np.clip(img/np.maximum(expect,0.005),0,2.5),aspect='auto',cmap='gray')
            
# plt.imsave("dr_dt image normalized by odom.png", np.clip(img/np.maximum(expect,0.005),0,2.5),cmap='gray')


"""
inputs:
dx_bot - bot fwd displace
dy_bot - bot sideways displace
dth_bot - bot spin displacement

outputs:
veth
ver
"""

3686.6302083333335
8642 25926
9075
max motion 1.0631881171052249
Help on function timeit in module timeit:

timeit(stmt='pass', setup='pass', timer=<built-in function perf_counter>, number=1000000, globals=None)
    Convenience function to create Timer object and call timeit method.



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [184]:
# %matplotlib widget
# fig = plt.figure()
from ipywidgets import interact, interact_manual
plt.plot([1],[1])
@interact(delt=(-4,4,.1),sc1=(500,len(raysx),1),sc2=(500,len(raysx),1))
def draw(delt=0, sc1 = 10, sc2 = 20):
    for artist in plt.gca().lines + plt.gca().collections:
        artist.remove()
    delt/=40
    sin=np.sin
    cos=np.cos
    ss=list(range(7888,7889))
    ss= [sc1]
    for scn in ss:
#         ax=plt.gca()
#         ax.set_xlim(-5,30)
#         ax.relim()
        x=raysx['ex'][scn]
        y=raysx['ey'][scn]
        delt+=1/40*1
        xo=np.interp(raysx['ts'][scn],odo['ts'],odo['x'])
        yo=np.interp(raysx['ts'][scn],odo['ts'],odo['y'])
        tho=np.interp(raysx['ts'][scn],odo['ts'],np.unwrap(odo['th']))
        delt -= 1/40*1
        xp = cos(tho)*x - sin(tho)*y + xo
        yp = sin(tho)*x + cos(tho)*y + yo
        good = raysx[scn]['r']>0
#         plt.plot(xp[good],yp[good],',')
        
        for scn2 in [sc2]:
#             xo=np.interp(raysx['ts'][scn2]+delt-1/40,odo['ts'],odo['x'])
#             yo=np.interp(raysx['ts'][scn2]+delt-1/40,odo['ts'],odo['y'])
#             tho=np.interp(raysx['ts'][scn2]+delt-1/40,odo['ts'],np.unwrap(odo['th']))
#             x2 = cos(tho)*(xp-xo) + sin(tho)*(yp-yo)
#             y2 = -sin(tho)*(xp-xo) + cos(tho)*(yp-yo) 
#             good = raysx[scn]['r']>0
#             #now we have scn in scn2
#             plt.plot(x2[good],y2[good],',')
            xo=np.interp(raysx['ts'][scn2]+delt,odo['ts'],odo['x'])
            yo=np.interp(raysx['ts'][scn2]+delt,odo['ts'],odo['y'])
            tho=np.interp(raysx['ts'][scn2]+delt,odo['ts'],np.unwrap(odo['th']))
            x2 = (cos(tho)*(xp-xo) + sin(tho)*(yp-yo))
            y2 = (-sin(tho)*(xp-xo) + cos(tho)*(yp-yo)) 
            good = (raysx[scn]['r']>0)
            #now we have scn in scn2
            plt.plot(x2[good],y2[good],'.')
#             xo=np.interp(raysx['ts'][scn2]+delt+1/40,odo['ts'],odo['x'])[:384]
#             yo=np.interp(raysx['ts'][scn2]+delt+1/40,odo['ts'],odo['y'])[:384]
#             tho=np.interp(raysx['ts'][scn2]+delt+1/40,odo['ts'],np.unwrap(odo['th']))[:384]
#             x2 = cos(tho)*(xp-xo) + sin(tho)*(yp-yo)
#             y2 = -sin(tho)*(xp-xo) + cos(tho)*(yp-yo) 
#             good = raysx[scn]['r']>0
#             #now we have scn in scn2
#             plt.plot(x2[good],y2[good],'o')
            odotmp=odo.copy()
            odotmp['ts']-=np.interp(raysx['ts'][scn2][-1]+delt,odo['ts'],odo['x'])
            t
            plt.plot(raysx['ex'][scn2][384:768],raysx['ey'][scn2][384:768],'x')
#     plt.ylim(-4,-10)
#     plt.xlim(-4,4)
#     plt.xlim(-40,-36)
#     plt.ylim(2.0,5.5)
      
    s=plt.gcf().canvas
    _, _, w, h = s.figure.bbox.bounds

    s.manager.resize(w, h)
# plt.plot

# plt.plot(raysx['ts'][7580:7995].ravel(), np.interp(raysx['ts'][7580:7995].ravel()+delt,odo['ts'],np.unwrap(odo['th'])),',')

interactive(children=(FloatSlider(value=0.0, description='delt', max=4.0, min=-4.0), IntSlider(value=500, desc…

In [183]:
fig = plt.figure()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [216]:
# Plan
"""
1. Read laser and odometry
2. Project local group of laser scans with odometry to one and two time steps forward
3. By block, select best time offset among both choices
4. Reproject using only best choices
5. Reselect by block from both choices
6. Repeat until convergence
"""

"""
val=[]
for k1 in range(-3,3,0.1):
    for k2 in range(-3,3,0.1):
        for fwd in range(0,6.28,0.3):
            

def offset(rays, dx0, dy0, dth(0)

def get_next_roto(rays,odo,interval):
    # gives the expected next positon of ray endpoints, given only rotation
    # takes in rays rotated so that the direction of motion is always +x
    derivative_width = 1/20
    dx = d_dt(ots, odo['x'], derivative_width)
    dy = d_dt(ots, odo['y'], derivative_width)
    ds = np.sqrt(dx**2 + dy**2)
    dth = d_dt(ots, np.unwrap(odo['th']), derivative_width)
    th_step = 1/4*np.pi/180
    raysout = rays.copy()
    for scan, scanout,dth2 in zip(rays.reshape(-1,1081), raysout.reshape(-1,1081), dth):
        thp = np.arange(len(scan))*th_step
        good = (scan['r']>0) & (scan['r']<40)
        nextth = np.clip(thp-dth2*interval, thp[0], thp[-1])
        nextx = np.interp(nextth,thp[good],scan['ex'][good])
        nexty = np.interp(nextth,thp[good],scan['ey'][good])
        scanout['ex'] = nextx
        scanout['ey'] = nexty
    return raysout
raysx=rays2
next_raysx = get_next_roto(raysx,odo,1/40)
dx = d_dt(ots, odo['x'], 1/40)
dy = d_dt(ots, odo['y'], 1/40)
ds = np.sqrt(dx**2 + dy**2)
@interact(fwddir=(0,6.28,0.01))
def tr(fwddir=0):
    plt.clf()
    scannum=235

    next_raysx = get_next_roto(raysx,odo,1/40)
    pred_scan = next_raysx.reshape(-1,1081)[scannum]
    nscan = raysx.reshape(-1,1081)[scannum+1]
    dss = ds[scannum]*1/40
    plt.plot(pred_scan['ex'],pred_scan['ey'],'g.')
    pred_scan['ex']+=dss*np.cos(fwddir)
    pred_scan['ey']+=dss*np.sin(fwddir)
#     print(dss)
    plt.plot(nscan['ex'],nscan['ey'],'b.')
    plt.plot(pred_scan['ex'],pred_scan['ey'],'r.')
    plt.xlim(-3,3)
    plt.ylim(-3,3)

# raysx=raysx[1081:]
# good=(raysx['r']>0)&(raysx['r']<40)&(next_raysx['r']>0)&(next_raysx['r']<30)
# err= np.sqrt((next_raysx['ey']-raysx['ey'])**2+(next_raysx['ey']-raysx['ey'])**2)
# plt.plot(raysx['th'][good],(err)[good],'b,')
"""
        
"""
def fwdize(rays, th):
    #rotates ray endpoints by th
    raysfwd=rays.copy()
    raysfwd['ex']=rays['ex']*np.cos(th)-rays['ey']*sin(th)
    raysfwd['ey']=rays['ex']*np.sin(th)+rays['ey']*cos(th)
    return raysfwd
sols=[]
for th in np.linspace(2*np.pi):
    rf1m=fwdize(rays1, th).reshape(-1,1081)[1:]
    rf1p=fwdize(rays1, th).reshape(-1,1081)[:-1]
    good = (rf1m['r']>0) & (rf1m['r']<40) & (rf1p['r']>0) & (rf1p['r']<40)
    plt.plot((rf1p['ex']-rf1m['ex'])
    rays1['ex']-rays[next_raysx = get_next_roto(raysx,odo,1/40)




def get_next(rays,odo,interval):
    # takes in rays rotated so that the direction of motion is always +x
    derivative_width = 1/20
    dx = d_dt(ots, odo['x'], derivative_width)
    dy = d_dt(ots, odo['y'], derivative_width)
    ds = np.sqrt(dx**2 + dy**2)
    dth = d_dt(ots, np.unwrap(odo['th']), derivative_width)
    th_step = 1/4*np.pi/180
    for scan in rays.reshape(-1,1081):
        thp = np.arange(len(scan))*th_step
        good = (scan['r']>0) & (scan['r']<40)
        nextth = np.clip(dth*interval, thp[0], thp[-1])
        nextx = np.interp(nextth,thp[good],scan['ex'][good]) + ds*interval
        nexty = np.interp(nextth,thp[good],scan['ey'][good])
        
"""
%matplotlib widget


"""
dx0=-ds*cos(fwd)
dy0=-ds*sin(fwd)
dx = dx0 + k1*dth0
dy = dy0 + k2*dth0
dth = dth0
dr = dx*cos(th) + dy*sin(th)
dth = -dx*sin(th)/r + dy*cos(th) - dth0
"""

def differentials(rays,ds,dth0,fwd,k1,k2):#k1 = y offset, k2 = x offset
#     th_step = 1/4*np.pi/180
#     th = np.arange(len(scan0))*th_step
    th=rays['th']
    r=rays['r']
    sin=np.sin
    cos=np.cos
    dx0=-ds*cos(fwd)
    dy0=-ds*sin(fwd)
    dx = dx0 + k1*dth0
    dy = dy0 - k2*dth0
    dth = dth0
    dr = dx*cos(th) + dy*sin(th)
    dth = -dx*sin(th)/r + dy*cos(th)/r - dth0
    return dr,dth

def mismatch(scan0, scan1, dr, dth):
    def get_nextr(interval):
        th_step = 1/4*np.pi/180
        assert len(scan0)==1081
        assert len(dth)==1081
#         thp = np.arange(len(scan0))*th_step
        thp = scan0['th']
        good = (scan0['r']>0.2) & (scan0['r']<40)
        nextth = np.clip(thp+dth*interval, thp[good][0], thp[good][-1])
        nextr = scan0['r']+dr*interval
        return nextth,nextr
    nextth1, nextr1=get_nextr(1)
    nextth2, nextr2=get_nextr(2)
    thp = scan1['th']
    good = (scan1['r']>0.2) & (scan1['r']<40)
    pb1=np.interp(nextth1,thp[good],scan1['r'][good])
    pb2=np.interp(nextth2,thp[good],scan1['r'][good])
    pseudoindex = np.interp(nextth2,thp[good], np.arange(len(scan1))[good])
    blockjump = (np.arange(len(scan1))+0.5)//384 != (pseudoindex+0.5)//384
    
    
    good = (scan0['r']>0) & (scan0['r']<40)&(scan1['r']>0) & (scan1['r']<40)
    use1=np.abs(nextr1-pb1)<=np.abs(nextr2-pb2)
    nextr=nextr2.copy()
    nextr[use1]=nextr1[use1]
    nextth=nextth2.copy()
    nextth[use1]=nextth1[use1]
    pb=pb2.copy()
    pb[use1]=pb1[use1]
#     th_step = 1/4*np.pi/180
#     thp=np.arange(len(scan0))*th_step
    thp=scan0['th']
#     plt.plot(thp,scan0['r'],'.')
#     plt.plot(scan1['th'],scan1['r'],'x')
    
#     plt.plot((nextth1)[good],nextr1[good],'o')
#     plt.plot((nextth2)[good],nextr2[good],'o')
#     plt.plot(pb2-nextr2)
#     plt.plot(pb1-nextr1)
#     plt.plot(np.arange(len(scan0))[good],use1[good],'.')
#     plt.plot(nextth[good],pb[good])
#     plt.plot(nextth[good],nextr[good])
#     plt.plot((pb-nextr)[good])
    indic = use1*2-1
    indic *= good
    indic = np.clip(np.abs(nextr1-pb1)-np.abs(nextr2-pb2),-0.05,0.05)
    return indic,(pb-nextr)+100*blockjump
#     plt.plot(thp,dr)
#     plt.plot(thp,nextr2)
ots = (odo['ts']-odo['ts'][0])
raysx=rays1.copy().reshape(-1,1081)
raysx['th']=np.arctan2(raysx['ey'],raysx['ex'])  # redefine angle
raysx['r'] = np.sqrt(raysx['ey']**2+raysx['ex']**2)*(raysx['r']>0) #redefine range
dx = d_dt(ots, odo['x'], 1/40)
dy = d_dt(ots, odo['y'], 1/40)
ds = dx*np.cos(odo['th'])+dy*np.sin(odo['th'])
dth = d_dt(ots, np.unwrap(odo['th']), 1/40)
# plt.plot(ds)
# plt.plot(np.cos(odo['th']))
# plt.plot(dy)
# plt.plot(odo['th'])
# plt.plot(np.arctan2(dy,dx))
scannum=325
# ds=ds[scannum]
# dth=dth[scannum]
# scan= rays1.reshape(-1,1081)[scannum]
ds=np.interp(raysx[:,-1]['ts'], odo['ts'],ds)
dth00=np.interp(raysx[:,-1]['ts'], odo['ts'],dth)

ds2=ds.reshape(-1,1)*np.ones_like(raysx['th'])
dth0=dth00.reshape(-1,1)*np.ones_like(raysx['th'])
print(ds2)
# assert False
v=-0.1
for fwd in [.0]:#np.linspace(0.5,0.8,10):
    print(fwd, end=' ')
    dr,dth = differentials(raysx,ds=ds2,dth0=dth0,fwd=fwd,k1=0,k2=0)
    
    jumps = []
    acc=[]
    for scannum in range(0,1000):#range(325,326): # 5138 has high turn rate
#         plt.title(str(dth0[scannum]/40))
# #         print(dr[scannum][0])
        scan0= raysx.reshape(-1,1081)[scannum]
        scan1= raysx.reshape(-1,1081)[scannum+1]
        jump, err = mismatch(scan0,scan1,dr[scannum]/40, dth[scannum]/40)
        jumps.append(jump)
        acc.append(err)
    jumps = np.array(jumps)
    jumps[np.isnan(jumps)]=0
    acc = np.array(acc)
    plt.imshow(acc>95)
    plt.figure()
    plt.title("Yellow = Long jump btwn scans. Blue = Short Jump. Turquoise = No Evidence")
    plt.imshow(jumps*(np.abs(np.clip(acc,-.1,0.1))<0.05))
    plt.ylabel("Scan Number")
    plt.xlabel("Sample Number")
    plt.figure()
    tes=jumps*(np.abs(np.clip(acc,-.1,0.1))<0.05) # best solution is within 5 cm
    tes0 = tes.copy()
    tes[:,:384]=np.roll(np.sort(tes[:,:384],axis=1),192,axis=1)
    tes[:,384:768]=np.roll(np.sort(tes[:,384:768],axis=1),192,axis=1)
    tes[:,768:]=np.roll(np.sort(tes[:,768:],axis=1),156,axis=1)
    tes[:,:384]=np.mean(tes[:,:384],axis=1).reshape(-1,1)
    tes[:,384:768]=np.mean(tes[:,384:768],axis=1).reshape(-1,1)
    tes[:,768:]=np.mean(tes[:,768:],axis=1).reshape(-1,1)
    plt.imshow(raysx[:2000,:]['i'])
    plt.figure()
    plt.imshow(np.hstack((tes*60,tes0*20)))
    plt.figure()
    plt.imshow(np.abs(np.clip(acc,-.1,0.1))<0.05)
#     acc=np.sort(np.hstack(acc))
#     plt.plot(acc)
    print(acc[100])
#     # plt.plot(np.arctan2(scan1['ey'],scan1['ex']))
#     plt.ylim(-1,1)
# #1.28 almost even
#0 big
    plt.figure()
    plt.plot(dth00/40)
    

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.0 

<ipython-input-216-f208ab9bbf55>:128: RuntimeWarning: invalid value encountered in true_divide
  dth = -dx*sin(th)/r + dy*cos(th)/r - dth0
<ipython-input-216-f208ab9bbf55>:149: RuntimeWarning: invalid value encountered in floor_divide
  blockjump = (np.arange(len(scan1))+0.5)//384 != (pseudoindex+0.5)//384
<ipython-input-216-f208ab9bbf55>:153: RuntimeWarning: invalid value encountered in less_equal
  use1=np.abs(nextr1-pb1)<=np.abs(nextr2-pb2)
<ipython-input-216-f208ab9bbf55>:222: RuntimeWarning: invalid value encountered in greater
  plt.imshow(acc>95)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-216-f208ab9bbf55>:225: RuntimeWarning: invalid value encountered in less
  plt.imshow(jumps*(np.abs(np.clip(acc,-.1,0.1))<0.05))


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[nan nan nan ... nan nan nan]


<ipython-input-216-f208ab9bbf55>:241: RuntimeWarning: invalid value encountered in less
  plt.imshow(np.abs(np.clip(acc,-.1,0.1))<0.05)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
%matplotlib widget
plt.plot(raysx[1000]['sx'])
plt.plot(raysx[0]['sy'])

        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
ure 1
# This cell checks odometry for smoothness
%matplotlib widget
def d_dt(t,x,dt,order=1):
    if order>1:
        x=d_dt(t,x,dt,order-1)
    if order==0:
        return x
    dt = dt/2
    lot = np.clip(t-0.001*dt, t[0], t[-1])
    hit = np.clip(t+dt, t[0], t[-1])
    assert(np.all(hit-lot > 0))
    lox = np.interp(lot,t,x)
    hix = np.interp(hit,t,x)
    return (hix-lox)/(hit-lot)
# first, resample to 100hz
odo_struct_rt = np.dtype(
    [
        ('ts', np.float64),
        ('x', np.float32),
        ('y', np.float32),
        ('th', np.float32),
        ('pad' , np.float32),
        ('lx', np.float32),
        ('ly', np.float32),
      ])
ots = (odo['ts']-odo['ts'][0])
hz=2
assert(np.all(np.diff(ots)>=0))
tsr = np.max(ots)-np.min(ots)
ts100 = np.linspace(ots[0],ots[-1],np.maximum(int(tsr*hz),2))
odo100 = np.zeros_like(ts100,dtype=odo_struct_rt)
odo100['ts'] = ts100
for key in ['x','y','th','lx','ly']:
    odo100[key]=np.interp(ts100,ots,odo[key])

for hz in [40,20,10,5,2]:
    # now check smoothness
#     plt.plot(np.unwrap(odo['th']))
    dk_dt = d_dt(ots,np.unwrap(odo['y']),1/hz,2)#/(d_dt(ots,np.unwrap(odo['th']),1/(2*hz))+0.1)
    plt.plot(ots,(dk_dt),)
    print(np.max(dk_dt))



SyntaxError: invalid syntax (<ipython-input-44-ca9e79a57758>, line 1)

In [61]:
%matplotlib widget

# odosub(odo)
# plt.plot(lp['ts'],np.sqrt(lp['x']**2+lp['y']**2), ',')
# plt.plot(lp['ts'],lp['th'], ',')

# plt.plot(rays0['ts'],np.sqrt(rays0['sx']**2+rays0['sy']**2), ',')
# plt.plot(rays0['ts'][647::1081],rays0['th'][647::1081], ',')
# plt.plot(np.arange(1081*20)%1081,rays0['r'][1081*200:1081*200+1081*20],',')

plt.plot(odo['ts'],odo['x'],'b')


# plt.plot(odo['x'],odo['y'], ',')
# ax1 = plt.subplot(211)
# ax2 = plt.subplot(212,sharex=ax1)
# ax1.plot(odo['ts'][:-1],np.diff(np.unwrap(odo['th'])), ',')
# ax2.plot(rays0['ts'][500::1081][:-1],(np.diff(rays0['th'][500::1081])+np.pi/4)%(np.pi/2)-np.pi/4, ',')

# ax2.plot(odo['ts'],odo['x'], ',')

# plt.plot(odo['ts'],np.unwrap(odo['th']),',')
# plt.plot(imu['ts'],np.unwrap((imu['th'])))
print(lp['th'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 0.         0.         0.        ... -0.7624614 -0.7518224 -0.7309431]


In [9]:
# As a hack to split scans up by sensor, we will look for sudden large jumps in the ray 
# starting position. As long as the localization doesn't jump too much, this hack should work.
# Plot the cdf of jumps to find a good cutoff.
# (as of this writing a 0.5m jump looks like a good cutoff)
%matplotlib widget

tmp=np.sqrt(np.diff(rays0['sx'],append=[0])**2+np.diff(rays0['sy'],append=[0])**2)
plt.plot(np.sort(tmp),',')

SENSOR_JUMP_THRESHOLD=0.5 # by inspection of the plot

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
# Use index to read sensor (uncomment block after if you want to use position hack instead
rays1=rays0[rays0['idx']==0]
rays2=rays0[rays0['idx']==1]

# # Use the jump threshold to detect changes of sensor
# restarts=np.sqrt(np.diff(rays0['sx'],append=[0])**2+np.diff(rays0['sy'],append=[0])**2)>SENSOR_JUMP_THRESHOLD

# # scans=rays0[:len(rays0)-len(rays0)%2162].reshape(-1,2162)
# raysr = rays0[restarts]# HACK to separate scans, gives the ray right before a restart
# idx=np.arange(len(restarts))[restarts]
# print("GCD of scan restarts:", np.gcd.reduce(idx[:-1]+1), 
#       "\nThis should be a multiple of the scan size if it's constant")
# rs=[]
# sn=[]
# n=0
# for s,e in zip(idx[:-1:2],idx[1::2]):
#     rays0[s+1:e+1]['idx']=0
#     rs.append(rays0[s+1:e+1])
#     sn.append(np.zeros(e-s)+n)
#     n+=1
# rays1=np.hstack(rs)
# scannum1=np.hstack(sn)
# # scannum1=scannum1[rays1['r']>=0]
# # rays1=rays1[rays1['r']>=0]

# rs=[rays0[:idx[0]+1]]
# sn=[]
# n=0
# for s,e in zip(idx[1::2],idx[2::2]):
#     rays0[s+1:e+1]['idx']=1
#     rs.append(rays0[s+1:e+1])
#     sn.append(np.zeros(e-s)+n)
#     assert(len(rays0[s+1:e+1])%1081==0)
#     n+=1

# rays2=np.hstack(rs)
# scannum2=np.hstack(sn)

# print("Timestamp of first ray:",rays1['ts'][0])

%matplotlib widget
# plt.plot(rays1['sx'],',')
# plt.plot(rays0['ts']-rays0['ts'][0],rays0['sx'],',')
# t=rays1['ts']-rays1['ts'][0]
# plt.plot(t,rays0['th']-300*t,',')
# plt.plot(rays1['ts'],np.mod(rays1['th']-2*np.pi*t/0.025025,2*np.pi),',')
# plt.plot(rays1['ts'],rays1['th'],',')
# plt.plot(rays1['ts'][540::1081],rays1['th'][540::1081],',')
# plt.plot(rays0['r']>0,',')
# _=plt.hist(np.diff(rays0['sx']),1000,range=[-.001,.001])
# plt.plot(np.arange(len(rays0))[restarts]%2162,',')
# plt.plot(np.diff(rays0[restarts]['ts'][::2]),',')

thtmp = rays1['th'].reshape(-1,1081)
rtmp = rays1['r'].reshape(-1,1081)



# print(rays2['th'].reshape(-1,1081)[:,540][0]-rays1['th'].reshape(-1,1081)[:,540][0])
c1x=rays1['ts'].reshape(-1,1081)[:,540]-rays1['ts'][0]
c1y=np.unwrap(thtmp[:,540])
c2x=rays2['ts'].reshape(-1,1081)[:,540]-rays2['ts'][0]
c2y=np.unwrap(np.mod(rays2['th'].reshape(-1,1081)[:,540]+3.145+np.pi,2*np.pi)-np.pi)
plt.plot((lp['ts']-lp['ts'][0]),lp['th']+c1y[0], 's')
plt.plot(c1x, c1y,'k.')
plt.plot(c2x, c2y,'r.')
plt.title("Plot of reference ray angle over time for both sensors")


plt.figure()
d1y=np.diff(c1y)/np.diff(c1x)
d1x=(c1x[:-1]+c1x[1:])/2
plt.plot(np.diff(d1y)/np.diff(d1x),'.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-45-f0d3a60601ae>:71: RuntimeWarning: invalid value encountered in true_divide
  d1y=np.diff(c1y)/np.diff(c1x)


In [11]:
%matplotlib widget
# plt.plot(rays1['ts'],rays1['sx'],',')
# plt.plot(rays2['ts'],rays2['sx'],',')

dx=np.diff(rays1['sx'])
dy=np.diff(rays1['sy'])
dd=np.sqrt(dx**2+dy**2)
dth=np.diff(rays1['th'])
dt=np.diff(rays1['ts'])
intertime=0.5*rays1['ts'][:-1]+0.5 * rays1['ts'][1:]


# plt.plot(intertime, dd/dt,',')
plt.plot( np.sort(dth/dt),',')
# plt.plot(rays1['ts'],rays1['th'],',')

# plt.plot(np.unwrap(rays1['th'])/np.pi,rays1['sx'],',')
# plt.plot(np.sort(np.diff(rays0['sx'])))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
## This cell tests for gyro scale error. 
#  If the plot shows a clear linear trend when GYRO_SCALE=1.0,
#  there is scale error. Adjust GYRO_SCALE until there is no 
#  clear linear trend. This corresponds to the amount the gyro 
#  needs to be multipled by.
#
# Input is "rays1" which is the rays from a single scanner.
# Right now it is hardcoded that a scan is 1081 rays.
# First scan is assumed to be stationary. If it's not, things get a little weird.

GYRO_SCALE=1.0 #0.95
GYRO_OFFSET = 0.0005#0.0014 # this number is 0.0005 even for perfect odometry! I wonder why?

%matplotlib widget
scans=rays1.reshape(-1,1081) #organize into scans

# Fill in the angles of the first scan to find offset angles of each ray
offth = scans['th']
tmp=scans['th'][0] # first scan angles
idxtmp=np.arange(len(tmp))[tmp!=0]
tmp2=tmp[tmp!=0] # first scan angles that are validish
    # Add in endpoints
slope=(tmp2[-1]-tmp2[0])/(idxtmp[-1]-idxtmp[0]) # Use the average slope to extrapolate endpoints
intercept=tmp2[0]-idxtmp[0]*slope
idxtmp=np.hstack((0,idxtmp,10000))
tmp2=np.hstack((intercept,tmp2,10000*slope+intercept))
offsets=np.interp(np.arange(len(tmp)),idxtmp,tmp2,left=0,right=0)

# Subtract the offsets to find the robot angle over tme
rotor=scans['th']-offsets
decent_angles=np.unwrap(rotor.ravel()[scans['r'].ravel()>0])
print(scans['ts'][0][0])
decent_ts=scans['ts'][scans['r']>0].ravel()
delt=np.diff(decent_angles,prepend=0)
slop=np.diff(decent_angles)/np.diff(decent_ts)
dupls=np.arange(len(slop))[np.diff(decent_ts)>.005] # jumps in timestamp indicating edge of scan
slop[dupls]=(slop[dupls+1]+slop[dupls-1])/2 # smooth out derivative at jumps
slop_rescale = (slop*GYRO_SCALE-GYRO_OFFSET)
incrs=slop_rescale*np.diff(decent_ts) # factor for correcting gyro scale error
regen_ang=np.cumsum(np.hstack((decent_angles[0],incrs)))

plt.figure()
plt.plot(offsets,'.')
plt.figure()
plt.plot(decent_ts,decent_angles,',')

plt.figure()
plt.plot(decent_angles,decent_angles-regen_ang)
plt.title("Adjust GYRO_SCALE to remove slope from this line")

plt.figure()
plt.plot(decent_ts,decent_angles)
plt.plot(decent_ts,regen_ang)
plt.title("Adjust GYRO_OFFSET to make flat areas in both lines flat")

plt.figure()
plt.plot(decent_ts,decent_angles-regen_ang)
# At end of cell:
# "decent_angles" is the angles from SLAM, using original odometry/IMU to do the slope within the scan
# "regen_ang" is the angles using only odometry/IMU, corrected for scale and bias
# "decent_ts" is the timestamps shared by both

1479662727.0105062


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
%matplotlib widget
# plt.plot(decent_ts,decent_angles)
plt.plot(decent_ts,regen_ang,',')

its = np.linspace(decent_ts[0],decent_ts[-1], 1.29*len(scans))
print("timestep",np.diff(its)[0])
iang = np.interp(its,decent_ts,decent_angles)
plt.plot(its,iang,'.')
     
kernel_size = 10
kernel = np.ones(kernel_size) / kernel_size
smoothed = np.convolve(np.hstack((iang,iang[::-1])), kernel, mode='same')[:len(iang)]#poor man's 'mirror' end conditions
smoothed2 = np.convolve(smoothed, kernel, mode='same')
smoothed3 = np.convolve(smoothed2, kernel, mode='same')
comp_smooth=smoothed2+(smoothed2-smoothed3)*2

# help(plt.hist)
plt.plot(its,smoothed)
# plt.plot(its,smoothed2)
# plt.plot(its,smoothed3)
# plt.plot(its,comp_smooth,'.')
# plt.plot(its,iang-comp_smooth)

# plt.hist(np.diff(smoothed)*10,1000,(-0.07,0.07))
# plt.hist(iang-smoothed,1000,(-0.07,0.07))
ira=np.interp(its,decent_ts,regen_ang)
# plt.plot(its,(ira-comp_smooth)*100)
# plt.plot(its,(ira-iang)*100)
plt.plot(lp['ts'],np.unwrap(lp['th'])-np.mean(np.unwrap(lp['th'])), markersize=2)


plt.figure()
tmp=(np.diff(smoothed,prepend=0)+np.diff(smoothed,append=smoothed[-1]))*10
# plt.plot(its,(np.diff(smoothed,prepend=0)+np.diff(smoothed,append=smoothed[-1]))*10)
plt.plot(its,iang-ira,'.')
plt.plot(its,np.diff(iang,prepend=0),'.')
# plt.plot(its,smoothed-ira)

# plt.plot(its,smoothed-ira -tmp*.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

timestep 0.024839401245117188


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# No idea what this cell did. Disabling.

# %matplotlib widget
# # plt.plot(decent_ts,decent_angles-regen_ang,',')

# slop_rate=.030/100
# tosum = np.hstack((decent_angles[0],incrs))
# dts = np.hstack((0,np.diff(decent_ts)))
# appr = regen_ang

# ang=0
# cors=[]
# angs=regen_ang.copy()
# for i in range(len(tosum)):
#     ang+=tosum[i]
#     corr=np.sign(decent_angles[i]-ang)*(dts[i]*slop_rate+0.1*np.abs(tosum[i]))
# #     print(decent_angles[i],ang,end=':')
#     ang+=corr
#     angs[i]=ang
# #     print(ang)
# fig, axs = plt.subplots(2,sharex=True)
# axs[0].plot(decent_ts,angs,',')
# axs[0].plot(decent_ts,decent_angles,',')
# axs[1].plot(decent_ts,np.cumsum(np.sign(decent_angles-angs)))
    
    
# # for i in range(10):

In [15]:
# No idea what this cell did. Disabling.

# %matplotlib widget
# for i in range(11):
#     corr = np.sign(decent_angles-appr)*dts*slop_rate

#     appr=np.cumsum(tosum+corr)
#     print(np.sum(np.abs(decent_angles-appr)))


# plt.plot(decent_ts,decent_angles-appr,',')
# print(np.sum(np.abs(decent_angles-appr)))
# # plt.plot(corr)


In [16]:
# No idea what this cell did. Disabling.

# %matplotlib widget

# plt.plot(decent_ts,np.hstack((0,slop_rescale)),',')

In [16]:
rays=rays0
print("orininal rays:\t",rays.shape)
r=rays['r']
keep=(r<10) & (r>0)
rays=rays[keep]
print("kept rays:\t",rays.shape)

orininal rays:	 (14716734,)
kept rays:	 (3545511,)


In [186]:
import ipywidgets
%matplotlib widget
from ipywidgets import interact, interact_manual
from IPython.display import display,clear_output
# rays2=rays1
# https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
# reveals issues: https://stackoverflow.com/questions/61715937/how-to-enable-matplotlib-blitting-to-create-fast-realtime-graph-in-jupyterlab
# 
number = 300
# plt.plot(rays['ex'][number*1081:(number+1)*1081],rays['ey'][number*1081:(number+1)*1081],'r,',c=[.5,.5,.5,.5])

@interact(number=(2,1000,1))
def display_scan(number=127):
    xlim=plt.gca().get_xlim()
    ylim=plt.gca().get_ylim()
    plt.gcf().clear()
    
#     plt.plot(raysy['ex'][number*1081:(number+1)*1081],raysy['ey'][number*1081:(number+1)*1081],'r,',c=[.5,.5,.5,.5])
    number -= 1
    plt.plot(raysy['ex'][number],raysy['ey'][number],'r.')
    number += 1
    plt.plot(raysy['ex'][number],raysy['ey'][number],'g.')
    number +=1 
    plt.plot(raysy['ex'][number],raysy['ey'][number],'b.')
    plt.plot(raysy['ex'][number][:768],raysy['ey'][number][:768],'c.')
    if raysy['ts'][number][-1]-raysy['ts'][number-1][-1]>1/30:
        plt.plot(raysy['ex'][number][:384],raysy['ey'][number][:384],'k')
        plt.title('Timestamp jump = Long (1)')
    else:
        plt.plot(raysy['ex'][number][:384],raysy['ey'][number][:384],'k')
        plt.title('Timestamp jump = Short (0)')
    number -=1
    plt.plot(raysy['sx'][number],raysy['sy'][number],'b.')

    if xlim[1]!=1:
        plt.xlim(xlim)
        plt.ylim(ylim)
#     plt.gcf().canvas.draw()
    s=plt.gcf().canvas
    _, _, w, h = s.figure.bbox.bounds
#     s._png_is_old = True
#     s.handle_refresh(1)
#     print(s._png_is_old)
    s.manager.resize(w, h)
    
#     plt.gcf().canvas.set_image_mode('full')
    

# plt.figure()
# for n in range(166,200):
#     plt.plot(raysy['ey'][n*1081:(n+1)*1081])
# plt.plot(raysy['ey'][198*1081:(198+1)*1081]) # weird jump at 383 to 384. Appears to repeat several data points (13?)


# DISCOVERY: The hokuyo has a glitch where sometimes the points [384:] are from one scan in the future relative to the early points!
# DISCOVERY2: Sometimes its the first points that go to the future!
# DISCOVERY3: It happens to both sensors
# DISCOVERY4: Actually there are at least 3 blocks of points [0:384][384:768][768:] that can jump to the future, as well as the timestamp. None of the 4 have a clear relation
# I wonder if the possibility of this is detectable from the timestamps? Like does it always happen before or after a missing scan? A: seems unpredictable :-(
# Also I notice that it can persist for a couple of scans, so that several in a row are split.

# Laser id 0
# Observations start at 2245
#                             a      a|
# # 5678901234567890123456789012345678|901234567890123456789
# S 0001100100110110011011010!00!00!!0|!!0000110110010011
# C 011001001000100010010?100100100010|0100110010010001001
# B 0101000100100100010010010010010001|001010100100010010 // Higher rays jumped (not sure, but mostly [384:768]
# A 1100010010001001000100100100100100|100010001001001000 // [0:384] jumped
# Y 0101001001001000100100100010010010|010011001000100100 // timestamp jumped

# Laser id 0
#  2590
#  90123456789012345678901234567890
#Y 101000010000100000100001000001
#C 101000010001000001000001000001
#B 100100001000001000010000010000
#A 100000100010000010000010000010



interactive(children=(IntSlider(value=127, description='number', max=1000, min=2), Output()), _dom_classes=('w…

In [16]:
# utility to find the ray nearest a certain point
def nearray(rays, x,y):
    return np.argmin((rays['ex']-x)**2+(rays['ey']-y)**2)

def plotcolored(x,y,phi):
    # plots multicolored data with phi indicating color angle
    rgb_cycle = np.vstack((            # Three sinusoids
    .5*(1.+np.cos(phi          )), # scaled to [0,1]
    .5*(1.+np.cos(phi+2*np.pi/3)), # 120° phase shifted.
    .5*(1.+np.cos(phi-2*np.pi/3)))).T
    idx = np.round(phi%(2*np.pi)/np.pi*30) #30 divisions for now
    for i in np.unique(idx):
        plt.plot(x[idx==i],y[idx==i],',',c=rgb_cycle[idx==i][0])

In [18]:
%matplotlib widget
cut=0
raysd=rays[(rays['i']>=cut)]#&(rays['idx']==1)]
span=540
# plt.plot(raysd['ex'][5925215-span:5925215+span],raysd['ey'][5925215-span:5925215+span],'r,')
# print(raysd['ex'][11706031])
plt.gca().set_facecolor('xkcd:black')
cycle_time = 10.0
phi=(raysd['ts']-raysd['ts'][0])*np.pi*2/cycle_time
rgb_cycle = np.vstack((            # Three sinusoids
    .5*(1.+np.cos(phi          )), # scaled to [0,1]
    .5*(1.+np.cos(phi+2*np.pi/3)), # 120° phase shifted.
    .5*(1.+np.cos(phi-2*np.pi/3)))).T
plotcolored(raysd['ex'],raysd['ey'],phi)
# line=plt.plot(raysd['ex'],raysd['ey'],',')#s=1,linewidth=0,c=rgb_cycle,antialiased=False)
# help(line[0])


# # Plot Odometry
# plt.plot(rays['sx'][::10],rays['sy'][::10],'r,')
# plt.plot(odo['x'],odo['y'], ',')
# plt.plot(lp['lx'],lp['ly'], 'c.')
# plt.gca().grid('on')
plt.gca().set_aspect('equal','box')

# #Plot a slice of interest
# s1=533456
# s2=534456
# plt.plot(rays1['ex'][s1:s2],rays1['ey'][s1:s2],'g.')
# plt.plot(rays1['sx'][s1:s2],rays1['sy'][s1:s2],'b.')


# plt.plot(raysd['ex'][slc:10000000],raysd['ey'][slc:10000000],'b,')


# plt.plot(rays1['sx'],rays1['sy'],'r,')
# plt.plot(raysd['sx'][1::2],raysd['sy'][1::2],'k,')
# plt.plot(raysr['sx'][::2],raysr['sy'][::2],'r.-')
# plt.plot(raysr['sx'][1::2],raysr['sy'][1::2],'k.-')
# plt.show()
# plt.figure()
# plt.plot(raysd['th'],'b,')

plt.figure()
ags=np.argsort(odo['ts'])

# plt.plot(odo['ts'][:-1],1/40*np.diff(odo['x'])/(np.diff(odo['ts'])),',')
plt.plot(lp['ts'][:],lp['th'],',')
# plt.plot(odo['ts'][:-1],1/40*np.diff(lp['th'])/(np.diff(lp['ts'])),'r,')
# plt.plot(odo['ts'],odo['th'])

print(nearray(raysd,16.08,-2.72))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

11829627


In [22]:
#Debug Odometry directly

%matplotlib widget

t=odo['ts']
x=odo['x']
y=odo['y']
th=odo['th']
ex = x - np.sin(th)*4
ey = y + np.cos(th)*4

plt.plot(ex,ey,'.')

ex = x + np.cos(th)*4
ey = y + np.sin(th)*4

plt.plot(ex,ey,'.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
## Find the max extent of the map
np.max(rays['ex'])
for c in ['ex','ey']:
    print(c, np.min(rays[c]),np.max(rays[c]), np.max(rays[c])-np.min(rays[c]))

ex -12.918765 29.284042 42.20281
ey -56.17813 7.3319387 63.51007


In [16]:
## Figure out rasterization
# Floor of float coords is integer coords
# Images are y flipped

def homp(T,pts):
    tmppts = pts @ T[:-1,:-1].T + T[:-1,-1]
    denom = pts @ T[-1:,:-1].T + T[-1,-1]
    return tmppts/denom


CELLS_PER_M = 100.0

formatter = {'float':lambda x:np.format_float_positional(x,precision=2,fractional=True,trim='-',pad_left=8,pad_right=2)}

def get_T_px_f_with_extent(rays, debug=False):
    # The corners in world space
    f_corns = np.array(
        [[np.min(rays['ex']),np.min(rays['ey'])],
         [np.max(rays['ex']),np.max(rays['ey'])]])
    T_pxc_f = np.array(
        [[1*CELLS_PER_M, 0             , 0],
         [0            , -1*CELLS_PER_M, 0],
         [0            , 0             , 1]])
    pxc_corns=homp(T_pxc_f,f_corns)
    pxc_topleft = np.array([  np.floor(np.min(pxc_corns[:,0])), np.floor(np.min(pxc_corns[:,1]))  ]) #inclusive
    pxc_btmright= np.array([  np.floor(np.max(pxc_corns[:,0])), np.floor(np.max(pxc_corns[:,1]))  ]) + 1 #exclusive
    pxc_extent = pxc_btmright-pxc_topleft

    T_px_f = T_pxc_f.copy()
    T_px_f[:-1,-1] = -pxc_topleft

    px_corns=homp(T_px_f,f_corns)
    px_topleft = np.array([  np.floor(np.min(px_corns[:,0])), np.floor(np.min(px_corns[:,1]))  ]) #inclusive
    px_btmright= np.array([  np.floor(np.max(px_corns[:,0])), np.floor(np.max(px_corns[:,1]))  ]) + 1 #exclusive
    px_extent = px_btmright-px_topleft

    assert (np.all(pxc_extent == px_extent))

    if debug:
        print("fcorns\n",f_corns)

        for var in ['pxc_corns',
                    'pxc_topleft',
                    'pxc_btmright',
                    'pxc_extent',
                    'px_corns',
                    'px_topleft',
                    'px_btmright',
                    'px_extent',
                    ]:
            print(var+'\n', eval(var))
    return T_px_f, px_extent.astype('int64')

with np.printoptions(formatter=formatter):
    T_px_f, px_extent = get_T_px_f_with_extent(rays, debug=True)

fcorns
 [[     -12.92      -56.18]
 [      29.28        7.33]]
pxc_corns
 [[   -1291.88     5617.81]
 [    2928.4      -733.19]]
pxc_topleft
 [   -1292        -734   ]
pxc_btmright
 [    2929        5618   ]
pxc_extent
 [    4221        6352   ]
px_corns
 [[       0.12     6351.81]
 [    4220.4         0.81]]
px_topleft
 [       0           0   ]
px_btmright
 [    4221        6352   ]
px_extent
 [    4221        6352   ]


In [17]:
# rasterize hits

def get_raster_coords(rays, T_px_f):
    # implicit inputs: T_px_f
    f_ept = np.vstack((rays['ex'].ravel(),rays['ey'].ravel())).T.copy()
    px_coords = homp(T_px_f, f_ept)
    px_coords = px_coords.reshape(rays['ex'].shape+(2,))
    return px_coords

px_coords = get_raster_coords(rays0, T_px_f)

bad0=rays0['r']<=0
bad0|=rays0['r']>59

img = np.zeros(px_extent[::-1])

# clip out bad pixels
bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
px_coords = px_coords[bad==0,:]




np.add.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), 1)
%matplotlib widget

imsc=img.copy()
imsc[img!=0]=np.log10(img[img!=0])+0.301



plt.imshow(img)
print('maxhits=',np.max(img))
plt.figure()
plt.plot(img[900,:],'.')
# plt.hist(rays0['r'],100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

maxhits= 931.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
# It turns out that gaussian blurring and selecting where >1 produces a very good approximation of where the walls are
# without much reflection. This cell does that calculation
%matplotlib widget

def produce_wall_mask(rays0):
    import cv2
    T_px_f, px_extent = get_T_px_f_with_extent(rays) # extents and transform
    px_coords = get_raster_coords(rays0, T_px_f) # pixel coordinates
    img = np.zeros(px_extent[::-1]) #image to fill

    # clip out bad rays
    bad0=rays0['r']<=0
    bad0|=rays0['r']>59
    # clip out bad pixels
    bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
    px_coords = px_coords[bad==0,:]

    np.add.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), 1)
    
    gaussian_func = lambda img:cv2.GaussianBlur(img,(61,61),sigmaX=3,borderType=cv2.BORDER_DEFAULT)

    local_mean = gaussian_func(img)
    return local_mean>1.0
wall_mask=produce_wall_mask(rays0)
plt.imshow(wall_mask)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
%matplotlib widget

def produce_fwhm_mask(rays0):
    import cv2
    T_px_f, px_extent = get_T_px_f_with_extent(rays) # extents and transform
    px_coords = get_raster_coords(rays0, T_px_f) # pixel coordinates
    img = np.zeros(px_extent[::-1]) #image to fill

    # clip out bad rays
    bad0=rays0['r']<=0
    bad0|=rays0['r']>59
    # clip out bad pixels
    bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
    px_coords = px_coords[bad==0,:]

    np.add.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), 1)
    
    kernel = np.ones((10,10), np.uint8)
    dil=cv2.dilate(img, kernel, borderType=cv2.BORDER_DEFAULT)
    plt.imshow((img>=dil/2)*(img>3))

    return (img>dil/2)
fwhm_mask=produce_fwhm_mask(rays0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
# load gt data and estimate normals
%matplotlib widget
import cv2

gtrgb=cv2.imread("gtmap.png")
# plt.imshow((gtrgb[:,:,1]>0)+0.5*fwhm_mask)
gtwall = gtrgb[:,:,1]>0
gtwall_coords = np.array(np.nonzero(gtwall))[::-1].T+0.5


# see the distribution of coordinates by distance
from scipy.spatial import KDTree
tree= KDTree(gtwall_coords)
d,nn=tree.query(px_coords[::360],distance_upper_bound=10.0)
# d,nn=tree.query(px_coords[:108100],distance_upper_bound=10.0)

# histogram out to +- 0.5 cell spacing is linear ramp, because it is an expanding circle. Beyond that it should approach gaussian in absense of localization error
# arclen=arcsin(CELL_SPACING/2/x)*4/(2*pi)*(2*pi*r)
# density = exp(-2*(x/SD)**2) # where x points in the normal direction
REGRESSION_SD=2.0
plt.hist(d[d<10],100)
x=np.linspace(0.01,10,1000)
plt.plot(x,np.arcsin(np.clip(0.5/x,0,1))*4*x*np.exp(-(x/REGRESSION_SD)**2)*1000)#1000 scale factor for visualization, adjust as needed

# plt.plot(px_coords[:108100,0],px_coords[:108100,1],',')
plt.figure()
plt.plot(gtwall_coords[:,0],gtwall_coords[:,1],',')
# x=np.linspace(0,10)
# plt.plot(x,np.exp(-(x/4)**2)*10000)
# print(nn)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
%matplotlib widget

def produce_max_inten_image(rays0):
    import cv2
    T_px_f, px_extent = get_T_px_f_with_extent(rays) # extents and transform
    px_coords = get_raster_coords(rays0, T_px_f) # pixel coordinates
    img = np.zeros(px_extent[::-1]) #image to fill

    # clip out bad rays
    bad0=rays0['r']<=0
    bad0|=rays0['r']>59
    bad0|=np.abs(rays0['th']+1.627-np.pi*3/2)>.0305
    # clip out bad pixels
    bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
    px_coords = px_coords[bad==0,:]
    
    inten = rays0['i'][bad==0]

    np.maximum.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), inten)
#     plt.imshow(img)
    f2=np.all(np.abs(np.floor(px_coords)-[2101,4168])<1000,axis=1) #top right:[3773,960] , mid left:[363,3871], alcove:[2018,2032], inner pillar:[2483,713]
    print(np.sum(f2))
    
    angs = np.interp(rays0['ts'],decent_ts,decent_angles)
#     plt.plot(rays0['ts'][bad==0][f2],(rays0['th'])[bad==0][f2],'.')
#     plt.plot(rays0['r'][bad==0][f2],rays0['i'][bad==0][f2],'.')
    plt.plot(rays0['th'][bad==0][f2],rays0['i'][bad==0][f2]+rays0['r'][bad==0][f2]*0,',')
#     plt.plot(rays0['r'][bad==0][f2],rays0['i'][bad==0][f2]+rays0['r'][bad==0][f2]*0,',')

#     plt.plot(rays0['ts'][bad==0][f2],rays0['i'][bad==0][f2],'.')
#     plt.plot(rays0['ts'][bad==0][f2],rays0['r'][bad==0][f2]*100,'.')
#     plt.plot(rays0['ts'][bad==0][f2],rays0['th'][bad==0][f2]*1000,'.')


#     plt.plot(rays0['i'][bad==0][f2],'.')

    plt.plot([2.37,7.93],[3105,1891])
    print(np.diff([3105,1891])/np.diff([2.37,7.93]))
    
    return img
maxinten=produce_max_inten_image(rays0)
plt.figure()
plt.imshow(maxinten>0)

14929


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[-218.34532374]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
#try some local enhancement:
%matplotlib widget

import cv2
gaussian_func = lambda img:cv2.GaussianBlur(img,(61,61),sigmaX=3,borderType=cv2.BORDER_DEFAULT)
DECENT_LINE_FRACTION = 0.7

local_mean = gaussian_func(img)

line_img = np.zeros((300,300))
line_img[150]=100
print("expected best for line:",100/np.max(gaussian_func(line_img)))
# plt.imshow(np.clip(img/(local_mean+1),0,10)>.8)
kernel_max = 1/np.max(img/(local_mean+0.0000001)) # appoximation of the center value of the gaussian kernel
gaussian_without_center=(local_mean-img*kernel_max)/(1-kernel_max)#mean estimate ignoring center pixel
denom=np.clip(local_mean,.0001,1e9) # considered gaussian_without_center, but nope 
qual_analog=np.clip(np.clip(img-np.sqrt(img),0,1e9)/denom,0,50)#lower bound of conf interval scales with sqrt
determiner = (qual_analog>0.7)*(img>0)
# plt.imshow((qual_analog>0.7) +(img>0))  
plt.imshow(img>0)
plt.figure()
plt.plot(img[630:645,1257],'.')
print(np.unravel_index(np.argmax(img),img.shape))
# plt.imshow(qual_analog>.7)
# plt.plot(np.sort(qual_analog.ravel()))
print("best observed:",np.max(img)/local_mean.ravel()[np.argmax(img)])
# plt.imshow((img>2)*0.5+(np.clip(img/(local_mean+1),0,10)>.8)*0.5)

expected best for line: 7.519884823893004


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(858, 1257)
best observed: 3.6316730282930405


In [23]:
# find local maxima for realigning scans
%matplotlib widget

kernels=np.array([
        [
            [0,0,1],
            [0,1,0],
            [1,0,0.0],
        ],
        [
            [0,1,0],
            [0,1,0],
            [0,1,0],
        ],
        [
            [1,0,0],
            [0,1,0],
            [0,0,1],
        ],
        [
            [0,0,0],
            [1,1,1],
            [0,0,0],
        ],
    ]).astype(np.uint8)
sol=np.zeros_like(local_mean)
gaussian_func_s = lambda img:cv2.GaussianBlur(img,(61,61),sigmaX=3,borderType=cv2.BORDER_DEFAULT)
ridgeimg=gaussian_func_s(img)
for kernel in kernels:
    sol+=(ridgeimg==cv2.dilate(ridgeimg, kernel, iterations=1))
plt.imshow(np.clip(sol,0,8)*(local_mean>0.5)+(img>20))
print("nonmax",np.sum(img>0),np.sum((img>0)*(sol>0)),np.sum(img>1),np.sum((img>1)*(sol>0)))
print(np.sum((sol>0)*(local_mean>0.5)))

cores=np.array(np.nonzero((sol>0)*(local_mean>0.5))).T+[0.5,0.5]

from scipy.spatial import KDTree

tree=KDTree(cores)
T_px_f, px_extent = get_T_px_f_with_extent(rays) # extents and transform
px_coords = get_raster_coords(rays0, T_px_f) # pixel coordinates

In [31]:
print(len(px_coords))

d,matchidx = tree.query(px_coords[:,::-1][:100000], eps=2,distance_upper_bound=20) # can only query 10k pts per second :-(
error = px_coords[:,::-1]-cores[matchidx]
plt.figure()
plt.hist(d[rays0['r']>0], )


18135937


IndexError: index 126014 is out of bounds for axis 0 with size 126014

In [22]:
def cimshow(img):
    dpi = 200 # Affects only tick and text sizes
    margin = 0.05/(np.max(img.shape)/100) # (5% of the width/height of the figure...)
    xpixels, ypixels = img.shape[1], img.shape[0]

    # Make a figure big enough to accomodate an axis of xpixels by ypixels
    # as well as the ticklabels, etc...
    figsize = (1 + margin) * ypixels / dpi, (1 + margin) * xpixels / dpi

    fig = plt.figure(figsize=figsize, dpi=dpi)
    # Make the axis the right size...
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])

    ax.imshow(img, interpolation='none')
    return ax

In [93]:
%matplotlib widget
import scipy.signal as signal

inten=rays1['i'].reshape(-1,1081)
plt.imshow(inten[:3000])

#(img>2)*0.5+(np.clip(img/(local_mean+1),0,10)>.8)*0.5
px_xy = get_raster_coords(rays2.reshape(-1,1081), T_px_f)
bad=rays2['r'].reshape(-1,1081)<=0
bad|=rays2['r'].reshape(-1,1081)>59
print(np.sum(bad))
bad|=px_xy[:,:,0]<0
bad|=px_xy[:,:,1]<0
# bad|=np.any(px_xy,axis=2)<0
print(np.sum(bad))
bad|=px_xy[:,:,0]>img.shape[1]
print(np.sum(bad))
bad|=px_xy[:,:,1]>img.shape[0]
print(np.sum(bad))


px_inds = tuple(np.rollaxis(np.floor(px_xy)[...,::-1].astype('int32'),-1))
# print(px_inds.shape)
flatinds = np.ravel_multi_index(px_inds,img.shape,mode='clip') # need to hand linearize to specify what to do with bad inds
img[0,:]=0
img[:,0]=0
img_samp = img.ravel()[flatinds].reshape(-1,1081)
mean_samp = local_mean.ravel()[flatinds].reshape(-1,1081)
detr_samp = determiner.ravel()[flatinds].reshape(-1,1081)

qual=np.clip(np.clip(img_samp-np.sqrt(img_samp),0,1e9)/(mean_samp+0.00001),0,10)
metric = np.sum(qual>1.0,axis=1)/np.sum(bad==False,axis=1)

plt.figure()
plt.plot(metric)
# plt.plot(np.sum(img_samp>10,axis=1))
# plt.plot(metric,'.')

cutoff=signal.order_filter(metric,np.ones(61),45)
metric_flattened = metric-cutoff
metric_flattened[:31]=metric_flattened[31]
metric_flattened[-31:]=metric_flattened[-31]
to_keep=metric_flattened
to_keep=to_keep.reshape(-1,1)+np.zeros(1081)
# print(to_keep[100:120,100:110])
plt.plot(cutoff)
# plt.plot(metric_flattened)
plt.plot(np.sort(metric))

# display filtered points
imf1=np.zeros_like(img)
# px_inds=np.array(px_inds)
# np.maximum.at(imf.ravel(), flatinds, (metric-signal.order_filter(metric,np.ones(21),5)).ravel())
# np.maximum.at(imf1.ravel(), flatinds[bad==False], -to_keep[bad==False])
np.maximum.at(imf1.ravel(), flatinds[bad==False], to_keep[bad==False]+1)

# np.maximum.at(imf1.ravel(), flatinds, to_keep==51)

# np.add.at(imf1.ravel(), flatinds, (-1.0*(to_keep<0)))
# np.add.at(imf1.ravel(), flatinds[bad==False], (1.0*(to_keep>0))[bad==False])
# np.add.at(imf1.ravel(), flatinds, 1.0)

im=np.clip(imf1,-100,1)+determiner*0.2
plt.figure()
plt.imshow(img/(local_mean+10))
ce=5112
plt.plot(px_xy[ce-1,:,0],px_xy[ce-1,:,1],'r.')
plt.plot(px_xy[ce,:,0],px_xy[ce,:,1],'.')
plt.plot(px_xy[ce+1,:,0],px_xy[ce+1,:,1],'.')

# cimshow(im[:2000,:5000])
print(metric_flattened[2198])
print(np.max(img/(local_mean+10)))
goodgrad=img/(local_mean+10)
cv2.imwrite('tmp.png',goodgrad/3.8*256)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1676189
1692366
1698652
1698657


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-0.10840484271694317
3.861872264348472


True

In [92]:
# rasterize hits by time

%matplotlib widget
colorang=rays0['ts']
colorvec=2*np.vstack([np.cos(colorang),np.sin(colorang)])
rv=[1,0.0]
gv=[np.cos(2/3*np.pi),np.sin(2/3*np.pi)]
bv=[np.cos(4/3*np.pi),np.sin(4/3*np.pi)]
th=np.linspace(0,7)
r=np.clip(np.dot(rv,colorvec),-1,1)*0.5+0.5
g=np.clip(np.dot(gv,colorvec),-1,1)*0.5+0.5
b=np.clip(np.dot(bv,colorvec),-1,1)*0.5+0.5
# plt.plot(r)
# plt.plot(g)
# plt.plot(b)


f_ept = np.vstack((rays0['ex'],rays0['ey'])).T.copy()
bad0=rays0['r']<=0
bad0|=rays0['r']>5

im3=np.zeros(list(px_extent[::-1])+[3])
px_coords = homp(T_px_f, f_ept)

px_xyrgb = np.hstack((px_coords,r[:,np.newaxis],g[:,np.newaxis],b[:,np.newaxis]))
bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
del px_coords

px_xyrgb = px_xyrgb[bad==0]
px_xy=np.floor(px_xyrgb[:,0:2]).astype('int64')

lindex = np.ravel_multi_index((px_xy[:,1],px_xy[:,0]), px_extent[::-1],mode='raise')

im3.ravel()[3*lindex]=px_xyrgb[:,2]
im3.ravel()[3*lindex+1]=px_xyrgb[:,3]
im3.ravel()[3*lindex+2]=px_xyrgb[:,4]

# plt.imshow(im3)
import cv2
cv2.imwrite('color_by_time.png',
            (np.concatenate(
                (im3[:,:,::-1]*255.9,255*np.ones(list(px_extent[::-1])+[1])),axis=2
            )*255).astype('uint16'))

True

In [25]:
# dump every second of data into a file
timedir='time_imgs'
start_time = np.min(rays0['ts'])
duration = np.ceil(np.max(rays0['ts'])-np.min(rays0['ts']))
import cv2

def produce_basic_coords(rays0):
    import cv2
    T_px_f, px_extent = get_T_px_f_with_extent(rays) # extents and transform
    px_coords = get_raster_coords(rays0, T_px_f) # pixel coordinates
    img = np.zeros(px_extent[::-1]) #image to fill

    # clip out bad rays
    bad0=rays0['r']<=0
    bad0|=rays0['r']>59
    # clip out bad pixels
    bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
    px_coords = px_coords[bad==0,:]
    return px_coords

img = np.zeros(px_extent[::-1],np.uint8) #image to fill
    
   
for i in range(int(duration)):
    name = '{}.png'.format(i)
    select=rays0['ts']>=start_time+i
    select&= rays0['ts']<start_time+i+1
    px_coords = produce_basic_coords(rays0[select])
    np.maximum.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), 255)
    print(name)
    cv2.imwrite(name, img)
    np.minimum.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), 0)

    

0.png
1.png
2.png
3.png
4.png
5.png
6.png
7.png
8.png
9.png
10.png
11.png
12.png
13.png
14.png
15.png
16.png
17.png
18.png
19.png
20.png
21.png
22.png
23.png
24.png
25.png
26.png
27.png
28.png
29.png
30.png
31.png
32.png
33.png
34.png
35.png
36.png
37.png
38.png
39.png
40.png
41.png
42.png
43.png
44.png
45.png
46.png
47.png
48.png
49.png
50.png
51.png
52.png
53.png
54.png
55.png
56.png
57.png
58.png
59.png
60.png
61.png
62.png
63.png
64.png
65.png
66.png
67.png
68.png
69.png
70.png
71.png
72.png
73.png
74.png
75.png
76.png
77.png
78.png
79.png
80.png
81.png
82.png
83.png
84.png
85.png
86.png
87.png
88.png
89.png
90.png
91.png
92.png
93.png
94.png
95.png
96.png
97.png
98.png
99.png
100.png
101.png
102.png
103.png
104.png
105.png
106.png
107.png
108.png
109.png
110.png
111.png
112.png
113.png
114.png
115.png
116.png
117.png
118.png
119.png
120.png
121.png
122.png
123.png
124.png
125.png
126.png
127.png
128.png
129.png
130.png
131.png
132.png
133.png
134.png
135.png
136.png
137.png
138.pn

In [ ]:
# BELOW HERE IS ALL EXPERIMENTAL STUFF
# TODO Cell: Display only the points that have a closer start point than any other start point 
# i.e. the point which is when the robot was closest to that spot.


%matplotlib widget
#as a simple approximation, lets see if using a certain offset foward and backward in time is okay
offset = 1000

def dist(pt,pts):
    return np.sqrt(np.sum((pt-pts)**2,axis=1))

sx = rays1['sx']#use rays only from one laser
sy = rays1['sy']
spt= np.vstack((sx,sy)).T

ex = rays1['ex']
ey = rays1['ey']
ept=np.vstack((ex,ey)).T

#remove far rays
rays11=rays1[dist(ept,spt)<10]
scannum11=scannum1[dist(ept,spt)<10]



#reunpack the ray that remain

sx = rays11['sx']#use rays only from one laser
sy = rays11['sy']
spt= np.vstack((sx,sy)).T

ex = rays11['ex']
ey = rays11['ey']
ept=np.vstack((ex,ey)).T

# # check that we are closer that
# fsx = sx[offset:]
# fsy = sy[offset:]
# bsx = sx[:-offset]
# bsy = sy[:-offset]
# cex = ex[offset//2:-offset//2]
# cey = ex[offset//2:-offset//2]


rsx=raysr['sx']
rsy= raysr['sy']
rpt= np.vstack((rsx,rsy)).T


probes = np.arange(0,len(sx),1000) # some points where we can get the true answer
# probes = [5025083]

def dist(pt,pts):
    return np.sqrt(np.sum((pt-pts)**2,axis=1))
sols=[] #the ones with a good closest approach
bsols=[] # the ones that never seem to have a good closest approach
stats=[]
for p in probes:
    pex=ex[p]
    pey=ey[p]
    ppt=np.array([pex,pey])
    d=dist(ppt, rpt)
    d=dist(ppt, spt)
    timecost=np.abs(scannum11[p]-scannum11)/300
#     plt.clf()
#     plt.plot(timecost)
#     plt.plot(d)
#     plt.plot(d+timecost)
    d3=dist(ppt,ept)#distance to original endpoint

#     break
    closest = np.argmin(d+timecost)
#     plt.plot(d)

#     print(closest,pex,pey)
    cpt=spt[closest]
    d2=dist(cpt,spt)#distance to best center
#     plt.plot(d2)
#     d3=dist(ppt,ept)#distance to original endpoint
#     plt.plot(d3+d2)
    cost=d3+d2
    remin=np.argmin(cost)
    print(p,remin,cost[remin])
    if cost[remin]<0.1:
        sols.append(remin)
    else:
#         plt.plot(d)
#         plt.plot(d3)
        bsols.append(remin)
#         break
    stats.append(cost[remin])
plt.plot(ex[sols],ey[sols],"b.")
plt.plot(ex[bsols],ey[bsols],"r.")

# rsx[closest],rsy[closest]

In [ ]:

%matplotlib widget
plt.close('all')

# plt.plot(rays11['ex'],rays11['ey'],'b,')

# tpti = 1000000
# tpt=np.array([rays11['ex'][tpti],rays11['ey'][tpti]])
# stpt=np.array([rays11['sx'][tpti],rays11['sy'][tpti]])
# # plt.plot(raysr['sx'][::2],raysr['sy'][::2],'r.-')
# plt.plot(raysr['sx'][1::2],raysr['sy'][1::2],'k.-')

# plt.plot(rays11['ex'][tpti],rays11['ey'][tpti],'r.')
# plt.plot(rays11['sx'][tpti],rays11['sy'][tpti],'g.')

# print(len(raysr),np.max(scannum11))
# dr=dist(stpt, rpt)
# iraysr = np.argmin(dr)
# print(iraysr,scannum11[tpti])
# plt.figure()
# plt.plot(dr)

# plt.figure()
dr=dist(spt,ept)
plt.plot(dr,'-')

def advance(line, marker, mask):
    left=line[:-1]
    line[1:]=np.minimum(left,line[1:])
    line[1:]=np.maximum(line[1:],mask[1:])
#     left[left<line[1:]]left[left<line[1:]]
    return line
# advance(dr,dr,dr-0.1)
line=dr.copy()
marker=dr
mask=dr-0.05
for i in range(100):
    line=advance(line, marker, mask)
line=line[::-1]
marker = marker[::-1]
mask = mask[::-1]
for i in range(100):
    line=advance(line, marker, mask)
line=line[::-1]
marker = marker[::-1]
mask = mask[::-1]
plt.plot(line,'-')
localmins = (line==dr)
plt.figure()
plt.plot(rays11['ex'],rays11['ey'],'b,')
plt.plot(rays11['ex'][localmins],rays11['ey'][localmins],'r.')




In [ ]:
%matplotlib widget

plt.plot(ex[sols],ey[sols],"b.")
plt.plot(ex[bsols],ey[bsols],"r.")
plt.plot(ex[probes],ey[probes],"g,")

In [ ]:
%matplotlib widget
plt.hist(stats,500)